In [37]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

### Customization Parameters: Allow modifications and link back to the stored parameters - update_parameters

In [38]:
# Initialize the dictionary to store selected parameters
selected_parameters = {}

# Define the function to update parameters
def update_parameters(param_name, widget):
    selected_parameters[param_name] = widget.value

# Create widgets for each parameter
zones_dropdown = widgets.Dropdown(options=['Global Indices', 'US Stocks', 'European Stocks'],description='Zones:')
underlyings_dropdown = widgets.Dropdown(options=['EU_STOXX50E', 'EU_SXXP', 'EU_SX7E'],description='Underlyings:')
vol_type_dropdown = widgets.Dropdown(options=['Moneyness', 'ATM', 'IV'],description='Vol Type:')
param_level_slider = widgets.IntSlider(value=100,min=0,max=200,step=10,description='Param Level:')
vol_mat_dropdown = widgets.IntSlider(value=1,min=1,max=12,step=1,description='Vol Mat (months):')
real_vol_mat_slider = widgets.IntSlider(value=21,min=1,max=252,step=1,description='Real Vol Mat (b-days):')
start_date_picker = widgets.DatePicker(description="Start Date")
end_date_picker = widgets.DatePicker(description="End Date")
call_strike_dropdown = widgets.FloatSlider(value=0.25,min=0.0,max=1.0,step=0.05,description="Call Strike:")
put_strike_dropdown = widgets.FloatSlider(value=0.25,min=0.0,max=1.0,step=0.05,description="Put Strike:")

# Update selected_parameters dict based on selection
zones_dropdown.observe(lambda change: update_parameters('Zone', zones_dropdown), names='value')
underlyings_dropdown.observe(lambda change: update_parameters('Underlying', underlyings_dropdown), names='value')
vol_type_dropdown.observe(lambda change: update_parameters('Vol Type', vol_type_dropdown), names='value')
param_level_slider.observe(lambda change: update_parameters('Param Level', param_level_slider), names='value')
vol_mat_dropdown.observe(lambda change: update_parameters('Vol Mat (months)', vol_mat_dropdown), names='value')
real_vol_mat_slider.observe(lambda change: update_parameters('Real Vol Mat', real_vol_mat_slider), names='value')
start_date_picker.observe(lambda change: update_parameters('Start Date', start_date_picker), names='value')
end_date_picker.observe(lambda change: update_parameters('End Date', end_date_picker), names='value')
call_strike_dropdown.observe(lambda change: update_parameters('Call Strike', call_strike_dropdown), names='value')
put_strike_dropdown.observe(lambda change: update_parameters('Put Strike', put_strike_dropdown), names='value')

In [39]:
# Display all widgets together
display(zones_dropdown, underlyings_dropdown, vol_type_dropdown, param_level_slider,vol_mat_dropdown, real_vol_mat_slider, start_date_picker, end_date_picker,call_strike_dropdown, put_strike_dropdown)

Dropdown(description='Zones:', options=('Global Indices', 'US Stocks', 'European Stocks'), value='Global Indic…

Dropdown(description='Underlyings:', options=('EU_STOXX50E', 'EU_SXXP', 'EU_SX7E'), value='EU_STOXX50E')

Dropdown(description='Vol Type:', options=('Moneyness', 'ATM', 'IV'), value='Moneyness')

IntSlider(value=100, description='Param Level:', max=200, step=10)

IntSlider(value=1, description='Vol Mat (months):', max=12, min=1)

IntSlider(value=21, description='Real Vol Mat (b-days):', max=252, min=1)

DatePicker(value=None, description='Start Date', step=1)

DatePicker(value=None, description='End Date', step=1)

FloatSlider(value=0.25, description='Call Strike:', max=1.0, step=0.05)

FloatSlider(value=0.25, description='Put Strike:', max=1.0, step=0.05)

### Result - Updated df when widgets are triggered

In [40]:
# This selected_parameters dictionary now reflects the current state of all widget inputs in real-time.
selected_parameters

{}

### App step 2 integration 

In [41]:
# Plot
# Function to create and update a plot based on parameters
def plot_graph(param1, param2, ax):
    x = np.linspace(0, 10, 100)
    y = param1 * np.sin(x) + param2 * np.cos(x)
    ax.clear()
    ax.plot(x, y)
    ax.set_title("Customized Graph")
    ax.set_xlabel("X-axis")
    ax.set_ylabel("Y-axis")


# Initialize the dictionary to store selected parameters for each plot
selected_parameters = {
    'plot_1': {'param1': 0.5, 'param2': 1.0},
    'plot_2': {'param1': 0.5, 'param2': 1.0},
    'plot_3': {'param1': 0.5, 'param2': 1.0},
    'plot_4': {'param1': 0.5, 'param2': 1.0}
}

In [42]:
# Function to initialize and display the main slide with four plots and checkboxes on top-right corner
def display_main_slide():
    with main_slide_display:
        clear_output(wait=True)

        # Create an HBox to hold the two rows of plots
        row1 = widgets.HBox([create_plot_with_checkbox('plot_1'), create_plot_with_checkbox('plot_2')])
        row2 = widgets.HBox([create_plot_with_checkbox('plot_3'), create_plot_with_checkbox('plot_4')])

        # Display the main slide with the two rows
        display(widgets.VBox([row1, row2]))

# Function to create a plot with a checkbox on the top-right corner
def create_plot_with_checkbox(plot_id):
    plot_box = widgets.Output()
    checkbox = widgets.Checkbox(description=f"{plot_id.capitalize()}", value=False, layout=widgets.Layout(width='auto'))

    # Attach observer to checkbox
    checkbox.observe(lambda change: handle_checkbox_change(change, plot_id), names='value')

    # Create a figure for the plot
    with plot_box:
        fig, ax = plt.subplots(figsize=(5, 4))
        plot_graph(selected_parameters[plot_id]['param1'], selected_parameters[plot_id]['param2'], ax)
        plt.show()

    # Position the checkbox on the top-right by combining with HBox and VBox layouts
    plot_with_checkbox = widgets.VBox([
        widgets.HBox([widgets.Label(""), checkbox], layout=widgets.Layout(justify_content='flex-end')),
        plot_box
    ])

    return plot_with_checkbox

# Function to handle checkbox change events
def handle_checkbox_change(change, plot_id):
    if change['new']:  # Only act if the checkbox is checked
        open_customization_window(plot_id)

# Function to open the customization window for a specific plot
def open_customization_window(plot_id):
    customization_window.children = []  # Clear previous widgets
    
    # Retrieve current parameters for the selected plot
    param1 = selected_parameters[plot_id]['param1']
    param2 = selected_parameters[plot_id]['param2']
    
    # Create customization widgets
    param1_slider = widgets.FloatSlider(value=param1, min=0.1, max=2.0, step=0.1, description='Param 1')
    param2_slider = widgets.FloatSlider(value=param2, min=0.1, max=2.0, step=0.1, description='Param 2')
    save_changes_checkbox = widgets.Checkbox(description="Save Changes", value=False)
    
    # Attach observers to widgets for dynamic updates and saving
    param1_slider.observe(lambda change: update_customization_plot(param1_slider.value, param2_slider.value), names='value')
    param2_slider.observe(lambda change: update_customization_plot(param1_slider.value, param2_slider.value), names='value')
    save_changes_checkbox.observe(lambda change: save_changes(change, plot_id, param1_slider, param2_slider), names='value')
    
    # Display the customization window with the initial plot and widgets
    customization_window.children = [widgets.VBox([param1_slider, param2_slider, save_changes_checkbox]), plot_output]
    update_customization_plot(param1_slider.value, param2_slider.value)  # Initial plot display

# Function to update the plot in the customization window dynamically
def update_customization_plot(param1, param2):
    with plot_output:
        clear_output(wait=True)
        fig, custom_ax = plt.subplots(figsize=(5, 4))
        plot_graph(param1, param2, custom_ax)
        plt.show()

# Function to save changes and update the main slide
def save_changes(change, plot_id, param1_slider, param2_slider):
    if change['new']:  # Only act if "Save Changes" is checked
        # Save the parameters to selected_parameters
        selected_parameters[plot_id]['param1'] = param1_slider.value
        selected_parameters[plot_id]['param2'] = param2_slider.value
        
        # Update the main slide and clear the customization window
        display_main_slide()
        customization_window.children = []  # Clear the customization window

In [43]:
# Output display for the main slide and customization window
main_slide_display = widgets.Output()
customization_window = widgets.VBox()  # Changed to VBox to hold widgets and plot separately
plot_output = widgets.Output()  # Separate output widget for the plot inside the customization window


# Display the main slide with four plots and the customization window output area
display_main_slide()
display(main_slide_display, customization_window)

Output()

VBox()

In [44]:
selected_parameters

{'plot_1': {'param1': 0.5, 'param2': 1.0},
 'plot_2': {'param1': 0.5, 'param2': 1.0},
 'plot_3': {'param1': 0.5, 'param2': 1.0},
 'plot_4': {'param1': 0.5, 'param2': 1.0}}

### App flexible graph name, param and position

In [45]:
# Configuration DataFrame for plots
plot_configurations = pd.DataFrame([
    {'uid': 'tm', 'function_name': 'plot_graph', 'params': {'param1': 0.5, 'param2': 1.0},
     'layout': {'slide': 1, 'position': 'top-left'}},
    {'uid': 'plot_2', 'function_name': 'plot_graph', 'params': {'param1': 1.0, 'param2': 0.5},
     'layout': {'slide': 1, 'position': 'top-right'}},
    {'uid': 'plot_3', 'function_name': 'plot_graph', 'params': {'param1': 0.8, 'param2': 0.8},
     'layout': {'slide': 1, 'position': 'bottom-left'}},
    {'uid': 'plot_4', 'function_name': 'plot_graph', 'params': {'param1': 1.2, 'param2': 0.6},
     'layout': {'slide': 1, 'position': 'bottom-right'}}
])

# Output display for the main slide and customization window
main_slide_display = widgets.Output()
customization_window = widgets.VBox()
plot_output = widgets.Output()

# Example plot function that takes dynamic parameters
def plot_graph(ax, **params):
    x = np.linspace(0, 10, 100)
    y = params['param1'] * np.sin(x) + params['param2'] * np.cos(x)
    ax.clear()
    ax.plot(x, y)
    ax.set_title("Customized Graph")
    ax.set_xlabel("X-axis")
    ax.set_ylabel("Y-axis")

# Function to initialize and display the main slide with dynamic plots and checkboxes
def display_main_slide():
    with main_slide_display:
        clear_output(wait=True)

        # Create two rows of plots
        row1 = widgets.HBox([create_plot_with_checkbox(config) for config in plot_configurations.iloc[:2].to_dict('records')])
        row2 = widgets.HBox([create_plot_with_checkbox(config) for config in plot_configurations.iloc[2:].to_dict('records')])

        # Display the main slide
        display(widgets.VBox([row1, row2]))

def create_plot_with_checkbox(config):
    plot_box = widgets.Output()
    # Remove the description text by setting it to an empty string
    checkbox = widgets.Checkbox(description="", value=False, layout=widgets.Layout(width='auto'))

    # Attach observer to checkbox
    checkbox.observe(lambda change: handle_checkbox_change(change, config['uid']), names='value')

    # Create a figure for the plot based on configuration parameters
    with plot_box:
        fig, ax = plt.subplots(figsize=(5, 4))
        plot_function = globals()[config['function_name']]
        plot_function(ax, **config['params'])
        plt.show()

    # Position the checkbox on the top-right by combining with HBox and VBox layouts
    plot_with_checkbox = widgets.VBox([
        widgets.HBox([widgets.Label(""), checkbox], layout=widgets.Layout(justify_content='flex-end')),
        plot_box
    ])

    return plot_with_checkbox

# Function to handle checkbox change events
def handle_checkbox_change(change, uid):
    if change['new']:  # Only act if the checkbox is checked
        open_customization_window(uid)

# Function to open the customization window for a specific plot
def open_customization_window(uid):
    customization_window.children = []  # Clear previous widgets
    
    # Retrieve plot configuration by UID
    config = plot_configurations.loc[plot_configurations['uid'] == uid].iloc[0]
    params = config['params']

    # Create customization widgets dynamically based on params in configuration
    sliders = []
    for param_name, param_value in params.items():
        slider = widgets.FloatSlider(value=param_value, min=0.1, max=2.0, step=0.1, description=param_name)
        slider.observe(lambda change, name=param_name: update_customization_plot(uid, name, change.new), names='value')
        sliders.append(slider)
    
    save_changes_checkbox = widgets.Checkbox(description="Save Changes", value=False)
    save_changes_checkbox.observe(lambda change: save_changes(change, uid), names='value')
    
    # Display the customization window with sliders and save checkbox
    customization_window.children = [widgets.VBox(sliders + [save_changes_checkbox]), plot_output]
    update_customization_plot(uid, None, None)  # Initial plot display

# Function to update the plot in the customization window dynamically
def update_customization_plot(uid, param_name, param_value):
    # Update configuration DataFrame with new parameter value
    if param_name and param_value is not None:
        plot_configurations.loc[plot_configurations['uid'] == uid, 'params'].iloc[0][param_name] = param_value
    
    # Get updated parameters and redraw plot
    params = plot_configurations.loc[plot_configurations['uid'] == uid, 'params'].iloc[0]
    with plot_output:
        clear_output(wait=True)
        fig, custom_ax = plt.subplots(figsize=(5, 4))
        plot_graph(custom_ax, **params)
        plt.show()

# Function to save changes and update the main slide
def save_changes(change, uid):
    if change['new']:  # Only act if "Save Changes" is checked
        # Update the main slide and clear the customization window
        display_main_slide()
        customization_window.children = []  # Clear the customization window

In [46]:
# Display the main slide with four plots and the customization window output area
display_main_slide()
display(main_slide_display, customization_window)

Output()

VBox()

In [47]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Configuration DataFrame for plots
plot_configurations = pd.DataFrame([
    {'uid': 'tm', 'function_name': 'plot_graph', 'params': {'param1': 0.5, 'param2': 1.0}, 'layout': {'slide': 1, 'position': 'top-left'}},
    {'uid': 'plot_2', 'function_name': 'plot_graph', 'params': {'param1': 1.0, 'param2': 0.5}, 'layout': {'slide': 1, 'position': 'top-right'}},
    {'uid': 'plot_3', 'function_name': 'plot_graph', 'params': {'param1': 0.8, 'param2': 0.8}, 'layout': {'slide': 1, 'position': 'bottom-left'}},
    {'uid': 'plot_4', 'function_name': 'plot_graph', 'params': {'param1': 1.2, 'param2': 0.6}, 'layout': {'slide': 1, 'position': 'bottom-right'}}
])

# Output display for the main slide and customization window
main_slide_display = widgets.Output()
customization_window = widgets.VBox()
plot_output = widgets.Output()

# Example plot function that takes dynamic parameters
def plot_graph(ax, **params):
    x = np.linspace(0, 10, 100)
    y = params['param1'] * np.sin(x) + params['param2'] * np.cos(x)
    ax.clear()
    ax.plot(x, y)
    ax.set_title("Customized Graph")
    ax.set_xlabel("X-axis")
    ax.set_ylabel("Y-axis")

# Function to initialize and display the main slide with dynamic plots and checkboxes
def display_main_slide():
    with main_slide_display:
        clear_output(wait=True)
        
        # Create two rows of plots, fetching updated configurations
        row1 = widgets.HBox([create_plot_with_checkbox(config) for config in plot_configurations.iloc[:2].to_dict('records')])
        row2 = widgets.HBox([create_plot_with_checkbox(config) for config in plot_configurations.iloc[2:].to_dict('records')])

        # Display the main slide
        display(widgets.VBox([row1, row2]))

def create_plot_with_checkbox(config):
    plot_box = widgets.Output()
    checkbox = widgets.Checkbox(description="", value=False, layout=widgets.Layout(width='auto'))

    # Attach observer to checkbox
    checkbox.observe(lambda change: handle_checkbox_change(change, config['uid']), names='value')

    # Create a figure for the plot based on configuration parameters
    with plot_box:
        fig, ax = plt.subplots(figsize=(5, 4))
        plot_function = globals()[config['function_name']]
        plot_function(ax, **config['params'])
        plt.show()

    # Position the checkbox on the top-right by combining with HBox and VBox layouts
    plot_with_checkbox = widgets.VBox([
        widgets.HBox([checkbox], layout=widgets.Layout(justify_content='flex-end')),
        plot_box
    ])

    return plot_with_checkbox

# Function to handle checkbox change events
def handle_checkbox_change(change, uid):
    if change['new']:  # Only act if the checkbox is checked
        open_customization_window(uid)

# Function to open the customization window for a specific plot
def open_customization_window(uid):
    customization_window.children = []  # Clear previous widgets
    
    # Retrieve plot configuration by UID
    config = plot_configurations[plot_configurations['uid'] == uid].iloc[0]
    params = config['params']

    # Create customization widgets dynamically based on params in configuration
    sliders = []
    for param_name, param_value in params.items():
        slider = widgets.FloatSlider(value=param_value, min=0.1, max=2.0, step=0.1, description=param_name)
        slider.observe(lambda change, name=param_name: update_customization_plot(uid, name, change.new), names='value')
        sliders.append(slider)
    
    save_changes_checkbox = widgets.Checkbox(description="Save Changes", value=False)
    save_changes_checkbox.observe(lambda change: save_changes(change, uid), names='value')
    
    # Display the customization window with sliders and save checkbox
    customization_window.children = [widgets.VBox(sliders + [save_changes_checkbox]), plot_output]
    update_customization_plot(uid, None, None)  # Initial plot display

# Function to update the plot in the customization window dynamically
def update_customization_plot(uid, param_name, param_value):
    # Update configuration DataFrame with new parameter value
    if param_name and param_value is not None:
        # Use .at[] to ensure the DataFrame is updated correctly
        idx = plot_configurations.index[plot_configurations['uid'] == uid].tolist()[0]
        plot_configurations.at[idx, 'params'][param_name] = param_value
    
    # Get updated parameters and redraw plot
    params = plot_configurations.loc[plot_configurations['uid'] == uid, 'params'].iloc[0]
    with plot_output:
        clear_output(wait=True)
        fig, custom_ax = plt.subplots(figsize=(5, 4))
        plot_graph(custom_ax, **params)
        plt.show()

# Function to save changes and update the main slide
def save_changes(change, uid):
    if change['new']:  # Only act if "Save Changes" is checked
        # Update the main slide and clear the customization window
        display_main_slide()
        customization_window.children = []  # Clear the customization window

# Initial display of the main slide
display_main_slide()

# Display widgets
display(main_slide_display)
display(customization_window)


Output()

VBox()

### PPT bar menu select view

In [48]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Configuration for multiple slides
slides = [
    {'uid': 'slide_1', 'plots': [
        {'uid': 'plot_1', 'function_name': 'plot_graph', 'params': {'param1': 0.5, 'param2': 1.0}, 'position': 'top-left'},
        {'uid': 'plot_2', 'function_name': 'plot_graph', 'params': {'param1': 1.0, 'param2': 0.5}, 'position': 'top-right'},
        {'uid': 'plot_3', 'function_name': 'plot_graph', 'params': {'param1': 0.8, 'param2': 0.8}, 'position': 'bottom-left'},
        {'uid': 'plot_4', 'function_name': 'plot_graph', 'params': {'param1': 1.2, 'param2': 0.6}, 'position': 'bottom-right'}
    ]},
    {'uid': 'slide_2', 'plots': [
        {'uid': 'plot_5', 'function_name': 'plot_graph', 'params': {'param1': 0.3, 'param2': 1.2}, 'position': 'top-left'},
        {'uid': 'plot_6', 'function_name': 'plot_graph', 'params': {'param1': 1.1, 'param2': 0.7}, 'position': 'top-right'},
        {'uid': 'plot_7', 'function_name': 'plot_graph', 'params': {'param1': 0.7, 'param2': 1.0}, 'position': 'bottom-left'},
        {'uid': 'plot_8', 'function_name': 'plot_graph', 'params': {'param1': 1.3, 'param2': 0.9}, 'position': 'bottom-right'}
    ]}
]

# Output display for the selected slide and sidebar for navigation thumbnails
main_slide_display = widgets.Output(layout=widgets.Layout(width='70%'))
sidebar = widgets.VBox(layout=widgets.Layout(width='30%', overflow_y='auto', max_height='600px', border='1px solid black', padding='5px'))
selected_slide_index = 0  # Start by showing the first slide

# Plotting function (example)
def plot_graph(ax, **params):
    x = np.linspace(0, 10, 100)
    y = params['param1'] * np.sin(x) + params['param2'] * np.cos(x)
    ax.clear()
    ax.plot(x, y)
    ax.set_title("Plot")
    ax.set_xlabel("X-axis")
    ax.set_ylabel("Y-axis")

# Function to render the main slide view based on the selected slide index
def render_slide(slide_index):
    slide = slides[slide_index]
    with main_slide_display:
        clear_output(wait=True)

        # Create a figure layout for the main slide display
        fig, axs = plt.subplots(2, 2, figsize=(10, 8))
        
        # Map plot positions to subplot indices
        position_map = {'top-left': (0, 0), 'top-right': (0, 1), 'bottom-left': (1, 0), 'bottom-right': (1, 1)}
        
        # Draw each plot on the slide
        for plot in slide['plots']:
            pos = position_map[plot['position']]
            plot_function = globals()[plot['function_name']]
            plot_function(axs[pos], **plot['params'])
        
        plt.tight_layout()
        plt.show()

# Function to create a thumbnail for each slide in the sidebar
def create_thumbnail(slide_index):
    slide = slides[slide_index]
    thumbnail = widgets.Output()
    with thumbnail:
        fig, axs = plt.subplots(2, 2, figsize=(3, 3))
        
        # Draw mini-plots for each plot in this slide
        position_map = {'top-left': (0, 0), 'top-right': (0, 1), 'bottom-left': (1, 0), 'bottom-right': (1, 1)}
        for plot in slide['plots']:
            pos = position_map[plot['position']]
            plot_function = globals()[plot['function_name']]
            plot_function(axs[pos], **plot['params'])
        
        plt.tight_layout()
        plt.close(fig)  # Close the figure to prevent duplicate display
        display(fig)
    
    # Checkbox to select this slide
    checkbox = widgets.Checkbox(description="Select", value=False, layout=widgets.Layout(width='auto'))
    
    # Callback to show the slide when checkbox is clicked
    def on_checkbox_change(change):
        if change['new']:  # Only act if the checkbox is checked
            show_slide(slide_index)
            checkbox.value = False  # Uncheck after selection to allow re-selection
    
    checkbox.observe(on_checkbox_change, names='value')
    
    # Return a VBox with thumbnail and checkbox at the bottom
    return widgets.VBox([thumbnail, checkbox], layout=widgets.Layout(margin='5px'))

# Function to show a specific slide when a checkbox is clicked
def show_slide(slide_index):
    global selected_slide_index
    selected_slide_index = slide_index
    render_slide(slide_index)

# Initialize sidebar with thumbnails for all slides
sidebar.children = [create_thumbnail(i) for i in range(len(slides))]

# Main layout with main slide display and scrollable sidebar on the right
main_layout = widgets.HBox([
    main_slide_display,
    sidebar
])

# Display the initial slide and the sidebar layout
render_slide(selected_slide_index)
display(main_layout)

### Merge

In [71]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Slide and Plot Configurations
slides = [
    {'uid': 'slide_1', 'plots': [
        {'uid': 'plot_1', 'function_name': 'plot_graph', 'params': {'param1': 0.5, 'param2': 1.0}, 'position': 'top-left'},
        {'uid': 'plot_2', 'function_name': 'plot_graph', 'params': {'param1': 1.0, 'param2': 0.5}, 'position': 'top-right'},
        {'uid': 'plot_3', 'function_name': 'plot_graph', 'params': {'param1': 0.8, 'param2': 0.8}, 'position': 'bottom-left'},
        {'uid': 'plot_4', 'function_name': 'plot_graph', 'params': {'param1': 1.2, 'param2': 0.6}, 'position': 'bottom-right'}
    ]},
    {'uid': 'slide_2', 'plots': [
        {'uid': 'plot_5', 'function_name': 'plot_graph', 'params': {'param1': 0.3, 'param2': 1.2}, 'position': 'top-left'},
        {'uid': 'plot_6', 'function_name': 'plot_graph', 'params': {'param1': 1.1, 'param2': 0.7}, 'position': 'top-right'},
        {'uid': 'plot_7', 'function_name': 'plot_graph', 'params': {'param1': 0.7, 'param2': 1.0}, 'position': 'bottom-left'},
        {'uid': 'plot_8', 'function_name': 'plot_graph', 'params': {'param1': 1.3, 'param2': 0.9}, 'position': 'bottom-right'}
    ]}
]

# Outputs for main slide display, sidebar, customization, and plot rendering
main_slide_display = widgets.Output(layout=widgets.Layout(width='60%', border='1px solid black'))
sidebar = widgets.VBox(layout=widgets.Layout(width='20%', overflow_y='auto', max_height='600px', border='1px solid black', padding='5px'))
customization_window = widgets.VBox(layout=widgets.Layout(width='20%', border='1px solid black'))
plot_output = widgets.Output()
selected_slide_index = 0
selected_plot_uid = None  # Tracks the current plot being customized
plot_checkboxes = []  # Track plot checkboxes globally to enforce single selection

# Plotting function
def plot_graph(ax, **params):
    x = np.linspace(0, 10, 100)
    y = params['param1'] * np.sin(x) + params['param2'] * np.cos(x)
    ax.clear()
    ax.plot(x, y)
    ax.set_title("Customized Graph")
    ax.set_xlabel("X-axis")
    ax.set_ylabel("Y-axis")

# Function to render a slide based on index and keep it visible
def render_slide(slide_index):
    slide = slides[slide_index]
    global plot_checkboxes
    plot_checkboxes = []  # Reset plot_checkboxes for the current slide
    
    with main_slide_display:
        clear_output(wait=True)
        fig, axs = plt.subplots(2, 2, figsize=(10, 8))
        position_map = {'top-left': (0, 0), 'top-right': (0, 1), 'bottom-left': (1, 0), 'bottom-right': (1, 1)}

        # Display plots with checkboxes for selection within the same layout
        for plot in slide['plots']:
            pos = position_map[plot['position']]
            plot_function = globals()[plot['function_name']]
            plot_function(axs[pos], **plot['params'])
            
            # Create a checkbox for each plot and store in plot_checkboxes list
            checkbox = widgets.Checkbox(description=f"Select {plot['uid']}", value=False, layout=widgets.Layout(width='auto'))
            checkbox.observe(lambda change, uid=plot['uid']: handle_plot_selection(change, slide_index, uid), names='value')
            plot_checkboxes.append(checkbox)

        plt.tight_layout()
        plt.show()

    # Display the plot checkboxes below the main slide display to keep them in view
    display(widgets.VBox(plot_checkboxes))

# Function to handle plot selection for customization, ensuring only one checkbox is selected at a time
def handle_plot_selection(change, slide_index, plot_uid):
    global selected_plot_uid
    if change['new']:  # Only act if a plot checkbox is checked
        # Uncheck other checkboxes to enforce single selection
        for checkbox in plot_checkboxes:
            if checkbox.description != f"Select {plot_uid}":
                checkbox.value = False
        
        # Set the selected plot UID and open customization window
        selected_plot_uid = plot_uid
        open_customization_window(slide_index, plot_uid)

# Function to open the customization window for the selected plot
def open_customization_window(slide_index, plot_uid):
    customization_window.children = []  # Clear previous widgets
    slide = slides[slide_index]
    plot = next(plot for plot in slide['plots'] if plot['uid'] == plot_uid)
    params = plot['params']

    sliders = []
    for param_name, param_value in params.items():
        slider = widgets.FloatSlider(value=param_value, min=0.1, max=2.0, step=0.1, description=param_name)
        slider.observe(lambda change, name=param_name: update_plot_params(slide_index, plot_uid, name, change.new), names='value')
        sliders.append(slider)

    save_changes_checkbox = widgets.Checkbox(description="Save Changes", value=False)
    save_changes_checkbox.observe(lambda change: save_customization(slide_index, plot_uid), names='value')
    customization_window.children = [widgets.VBox(sliders + [save_changes_checkbox]), plot_output]
    update_plot_preview(slide_index, plot_uid)

# Update plot preview in the customization window
def update_plot_preview(slide_index, plot_uid):
    plot = next(plot for plot in slides[slide_index]['plots'] if plot['uid'] == plot_uid)
    with plot_output:
        clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(5, 4))
        plot_graph(ax, **plot['params'])
        plt.show()

# Save customization and refresh main view
def save_customization(slide_index, plot_uid):
    render_slide(slide_index)  # Rerender the main slide view with updated parameters

# Update plot parameters in the slides list and refresh preview
def update_plot_params(slide_index, plot_uid, param_name, param_value):
    plot = next(plot for plot in slides[slide_index]['plots'] if plot['uid'] == plot_uid)
    plot['params'][param_name] = param_value
    update_plot_preview(slide_index, plot_uid)

# Initialize sidebar with thumbnails for each slide, each with a slide selection checkbox
def create_thumbnail(slide_index):
    thumbnail = widgets.Output()
    with thumbnail:
        fig, axs = plt.subplots(2, 2, figsize=(3, 3))
        position_map = {'top-left': (0, 0), 'top-right': (0, 1), 'bottom-left': (1, 0), 'bottom-right': (1, 1)}
        for plot in slides[slide_index]['plots']:
            pos = position_map[plot['position']]
            plot_function = globals()[plot['function_name']]
            plot_function(axs[pos], **plot['params'])
        plt.tight_layout()
        plt.close(fig)
        display(fig)

    # Add checkbox to select this slide
    checkbox = widgets.Checkbox(description=f"Select Slide {slide_index + 1}", value=False, layout=widgets.Layout(width='auto'))
    checkbox.observe(lambda change: show_slide(change, slide_index), names='value')
    return widgets.VBox([thumbnail, checkbox], layout=widgets.Layout(margin='5px'))

# Show slide when selected in the sidebar
def show_slide(change, slide_index):
    if change['new']:
        render_slide(slide_index)

# Display main layout with sidebar and main slide display
def display_main_layout():
    sidebar.children = [create_thumbnail(i) for i in range(len(slides))]
    render_slide(selected_slide_index)  # Display the initial slide
    display(widgets.HBox([sidebar, main_slide_display, customization_window]))  # Ensure all sections are visible

# Initial display
display_main_layout()


# DEV

# mmm

In [77]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Slide and Plot Configurations
slides = [
    {'uid': 'slide_1', 'plots': [
        {'uid': 'plot_1', 'function_name': 'plot_graph', 'params': {'param1': 0.5, 'param2': 1.0}, 'position': 'top-left'},
        {'uid': 'plot_2', 'function_name': 'plot_graph', 'params': {'param1': 1.0, 'param2': 0.5}, 'position': 'top-right'},
        {'uid': 'plot_3', 'function_name': 'plot_graph', 'params': {'param1': 0.8, 'param2': 0.8}, 'position': 'bottom-left'},
        {'uid': 'plot_4', 'function_name': 'plot_graph', 'params': {'param1': 1.2, 'param2': 0.6}, 'position': 'bottom-right'}
    ]},
    {'uid': 'slide_2', 'plots': [
        {'uid': 'plot_5', 'function_name': 'plot_graph', 'params': {'param1': 0.3, 'param2': 1.2}, 'position': 'top-left'},
        {'uid': 'plot_6', 'function_name': 'plot_graph', 'params': {'param1': 1.1, 'param2': 0.7}, 'position': 'top-right'},
        {'uid': 'plot_7', 'function_name': 'plot_graph', 'params': {'param1': 0.7, 'param2': 1.0}, 'position': 'bottom-left'},
        {'uid': 'plot_8', 'function_name': 'plot_graph', 'params': {'param1': 1.3, 'param2': 0.9}, 'position': 'bottom-right'}
    ]}
]

# Outputs for main slide display, sidebar, customization, and plot rendering
main_slide_display = widgets.Output(layout=widgets.Layout(width='60%', border='1px solid black'))
sidebar = widgets.VBox(layout=widgets.Layout(width='20%', overflow_y='auto', max_height='600px', border='1px solid black', padding='5px'))
customization_window = widgets.VBox(layout=widgets.Layout(width='20%', border='1px solid black'))
plot_output = widgets.Output()
selected_slide_index = 0
selected_plot_uid = None  # Tracks the current plot being customized
plot_checkboxes = []  # Track plot checkboxes globally to enforce single selection

# Plotting function
def plot_graph(ax, **params):
    x = np.linspace(0, 10, 100)
    y = params['param1'] * np.sin(x) + params['param2'] * np.cos(x)
    ax.plot(x, y)
    ax.set_title("Customized Graph")
    ax.set_xlabel("X-axis")
    ax.set_ylabel("Y-axis")

# Function to render a slide with checkboxes at the top-right of each plot
def render_slide(slide_index):
    slide = slides[slide_index]
    global plot_checkboxes
    plot_checkboxes = []  # Reset plot_checkboxes for the current slide
    
    with main_slide_display:
        clear_output(wait=True)
        
        # Create grid layout for plots
        plot_grid = widgets.GridBox(
            children=[],
            layout=widgets.Layout(
                grid_template_columns="repeat(2, 1fr)",
                grid_gap="10px",
                align_items="center"
            )
        )

        # Display plots with checkboxes for selection within the same layout
        for plot in slide['plots']:
            plot_output = widgets.Output()
            checkbox = widgets.Checkbox(value=False, description=f"Select {plot['uid']}", indent=False, layout=widgets.Layout(width='auto'))
            checkbox.observe(lambda change, uid=plot['uid']: handle_plot_selection(change, slide_index, uid), names='value')
            plot_checkboxes.append(checkbox)

            with plot_output:
                fig, ax = plt.subplots(figsize=(5, 4))
                plot_graph(ax, **plot['params'])
                plt.show()

            # Arrange checkbox to appear above each plot on the top-right
            plot_box = widgets.VBox([
                widgets.HBox([checkbox], layout=widgets.Layout(justify_content='flex-end')),  # Checkbox aligned right
                plot_output
            ])
            plot_grid.children += (plot_box,)

        display(plot_grid)

# Function to handle plot selection for customization, ensuring only one checkbox is selected at a time
def handle_plot_selection(change, slide_index, plot_uid):
    global selected_plot_uid
    if change['new']:  # Only act if a plot checkbox is checked
        # Uncheck other checkboxes to enforce single selection
        for checkbox in plot_checkboxes:
            if checkbox.description != f"Select {plot_uid}":
                checkbox.value = False
        
        # Set the selected plot UID and open customization window
        selected_plot_uid = plot_uid
        open_customization_window(slide_index, plot_uid)

# Function to open the customization window for the selected plot
def open_customization_window(slide_index, plot_uid):
    customization_window.children = []  # Clear previous widgets
    slide = slides[slide_index]
    plot = next(plot for plot in slide['plots'] if plot['uid'] == plot_uid)
    params = plot['params']

    sliders = []
    for param_name, param_value in params.items():
        slider = widgets.FloatSlider(value=param_value, min=0.1, max=2.0, step=0.1, description=param_name)
        slider.observe(lambda change, name=param_name: update_plot_params(slide_index, plot_uid, name, change.new), names='value')
        sliders.append(slider)

    save_changes_checkbox = widgets.Checkbox(description="Save Changes", value=False)
    save_changes_checkbox.observe(lambda change: save_customization(slide_index, plot_uid), names='value')
    customization_window.children = [widgets.VBox(sliders + [save_changes_checkbox]), plot_output]
    update_plot_preview(slide_index, plot_uid)

# Update plot preview in the customization window
def update_plot_preview(slide_index, plot_uid):
    plot = next(plot for plot in slides[slide_index]['plots'] if plot['uid'] == plot_uid)
    with plot_output:
        clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(5, 4))
        plot_graph(ax, **plot['params'])
        plt.show()

# Save customization and refresh main view
def save_customization(slide_index, plot_uid):
    render_slide(slide_index)  # Rerender the main slide view with updated parameters

# Update plot parameters in the slides list and refresh preview
def update_plot_params(slide_index, plot_uid, param_name, param_value):
    plot = next(plot for plot in slides[slide_index]['plots'] if plot['uid'] == plot_uid)
    plot['params'][param_name] = param_value
    update_plot_preview(slide_index, plot_uid)

# Initialize sidebar with thumbnails for each slide, each with a slide selection checkbox
def create_thumbnail(slide_index):
    thumbnail = widgets.Output()
    with thumbnail:
        fig, axs = plt.subplots(2, 2, figsize=(3, 3))
        position_map = {'top-left': (0, 0), 'top-right': (0, 1), 'bottom-left': (1, 0), 'bottom-right': (1, 1)}
        for plot in slides[slide_index]['plots']:
            pos = position_map[plot['position']]
            plot_graph(axs[pos], **plot['params'])
        plt.tight_layout()
        plt.close(fig)
        display(fig)

    # Add checkbox to select this slide
    checkbox = widgets.Checkbox(description=f"Select Slide {slide_index + 1}", value=False, layout=widgets.Layout(width='auto'))
    checkbox.observe(lambda change: show_slide(change, slide_index), names='value')
    return widgets.VBox([thumbnail, checkbox], layout=widgets.Layout(margin='5px'))

# Show slide when selected in the sidebar
def show_slide(change, slide_index):
    if change['new']:
        render_slide(slide_index)

# Display main layout with sidebar and main slide display
def display_main_layout():
    sidebar.children = [create_thumbnail(i) for i in range(len(slides))]
    render_slide(selected_slide_index)  # Display the initial slide
    display(widgets.HBox([sidebar, main_slide_display, customization_window]))  # Ensure all sections are visible

# Initial display
display_main_layout()

In [75]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Slide and Plot Configurations
slides = [
    {'uid': 'slide_1', 'plots': [
        {'uid': 'plot_1', 'function_name': 'plot_graph', 'params': {'param1': 0.5, 'param2': 1.0, 'color': 'blue'}, 'position': 'top-left'},
        {'uid': 'plot_2', 'function_name': 'plot_graph', 'params': {'param1': 1.0, 'param2': 0.5, 'color': 'green'}, 'position': 'top-right'},
        {'uid': 'plot_3', 'function_name': 'plot_graph', 'params': {'param1': 0.8, 'param2': 0.8, 'color': 'red'}, 'position': 'bottom-left'},
        {'uid': 'plot_4', 'function_name': 'plot_graph', 'params': {'param1': 1.2, 'param2': 0.6, 'color': 'purple'}, 'position': 'bottom-right'}
    ]},
    {'uid': 'slide_2', 'plots': [
        {'uid': 'plot_5', 'function_name': 'plot_graph', 'params': {'param1': 0.3, 'param2': 1.2, 'color': 'orange'}, 'position': 'top-left'},
        {'uid': 'plot_6', 'function_name': 'plot_graph', 'params': {'param1': 1.1, 'param2': 0.7, 'color': 'cyan'}, 'position': 'top-right'},
        {'uid': 'plot_7', 'function_name': 'plot_graph', 'params': {'param1': 0.7, 'param2': 1.0, 'color': 'magenta'}, 'position': 'bottom-left'},
        {'uid': 'plot_8', 'function_name': 'plot_graph', 'params': {'param1': 1.3, 'param2': 0.9, 'color': 'yellow'}, 'position': 'bottom-right'}
    ]}
]

# Outputs for main slide display, sidebar, customization, and plot rendering
main_slide_display = widgets.Output(layout=widgets.Layout(width='60%', border='1px solid black'))
sidebar = widgets.VBox(layout=widgets.Layout(width='20%', overflow_y='auto', max_height='600px', border='1px solid black', padding='5px'))
customization_window = widgets.VBox(layout=widgets.Layout(width='20%', border='1px solid black'))
plot_output = widgets.Output()
selected_slide_index = 0
selected_plot_uid = None  # Tracks the current plot being customized
plot_checkboxes = []  # Track plot checkboxes globally to enforce single selection
slide_checkboxes = []  # Track slide checkboxes globally to enforce single selection

# Plotting function
def plot_graph(ax, **params):
    x = np.linspace(0, 10, 100)
    y = params['param1'] * np.sin(x) + params['param2'] * np.cos(x)
    ax.plot(x, y, color=params.get('color', 'blue'))
    ax.set_title("Customized Graph")
    ax.set_xlabel("X-axis")
    ax.set_ylabel("Y-axis")

# Function to render a slide with checkboxes at the top-right of each plot
def render_slide(slide_index):
    slide = slides[slide_index]
    global plot_checkboxes
    plot_checkboxes = []  # Reset plot_checkboxes for the current slide
    
    with main_slide_display:
        clear_output(wait=True)
        
        # Create grid layout for plots
        plot_grid = widgets.GridBox(
            children=[],
            layout=widgets.Layout(
                grid_template_columns="repeat(2, 1fr)",
                grid_gap="10px",
                align_items="center"
            )
        )

        # Display plots with checkboxes for selection within the same layout
        for plot in slide['plots']:
            plot_output = widgets.Output()
            checkbox = widgets.Checkbox(value=False, description=f"Select {plot['uid']}", indent=False, layout=widgets.Layout(width='auto'))
            checkbox.observe(lambda change, uid=plot['uid']: handle_plot_selection(change, slide_index, uid), names='value')
            plot_checkboxes.append(checkbox)

            with plot_output:
                fig, ax = plt.subplots(figsize=(5, 4))
                plot_graph(ax, **plot['params'])
                plt.show()

            # Arrange checkbox to appear above each plot on the top-right
            plot_box = widgets.VBox([
                widgets.HBox([checkbox], layout=widgets.Layout(justify_content='flex-end')),  # Checkbox aligned right
                plot_output
            ])
            plot_grid.children += (plot_box,)

        display(plot_grid)

# Function to handle plot selection for customization, ensuring only one checkbox is selected at a time
def handle_plot_selection(change, slide_index, plot_uid):
    global selected_plot_uid
    if change['new']:  # Only act if a plot checkbox is checked
        # Uncheck other checkboxes to enforce single selection
        for checkbox in plot_checkboxes:
            if checkbox.description != f"Select {plot_uid}":
                checkbox.value = False
        
        # Set the selected plot UID and open customization window
        selected_plot_uid = plot_uid
        open_customization_window(slide_index, plot_uid)

# Function to open the customization window for the selected plot
def open_customization_window(slide_index, plot_uid):
    customization_window.children = []  # Clear previous widgets
    slide = slides[slide_index]
    plot = next(plot for plot in slide['plots'] if plot['uid'] == plot_uid)
    params = plot['params']

    sliders = []
    for param_name, param_value in params.items():
        if param_name != 'color':  # Only add sliders for numeric params
            slider = widgets.FloatSlider(value=param_value, min=0.1, max=2.0, step=0.1, description=param_name)
            slider.observe(lambda change, name=param_name: update_plot_params(slide_index, plot_uid, name, change.new), names='value')
            sliders.append(slider)

    # Color picker widget
    color_picker = widgets.ColorPicker(
        value=params.get('color', 'blue'),
        description='Color',
        layout=widgets.Layout(width='auto')
    )
    color_picker.observe(lambda change: update_plot_params(slide_index, plot_uid, 'color', change.new), names='value')

    save_changes_checkbox = widgets.Checkbox(description="Save Changes", value=False)
    save_changes_checkbox.observe(lambda change: save_customization(slide_index, plot_uid), names='value')
    customization_window.children = [widgets.VBox(sliders + [color_picker, save_changes_checkbox]), plot_output]
    update_plot_preview(slide_index, plot_uid)

# Update plot preview in the customization window
def update_plot_preview(slide_index, plot_uid):
    plot = next(plot for plot in slides[slide_index]['plots'] if plot['uid'] == plot_uid)
    with plot_output:
        clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(5, 4))
        plot_graph(ax, **plot['params'])
        plt.show()

# Save customization and refresh main view and sidebar thumbnails
def save_customization(slide_index, plot_uid):
    render_slide(slide_index)  # Rerender the main slide view with updated parameters
    # Update the thumbnail for the current slide to reflect changes
    sidebar.children[slide_index] = create_thumbnail(slide_index)

# Update plot parameters in the slides list and refresh preview
def update_plot_params(slide_index, plot_uid, param_name, param_value):
    plot = next(plot for plot in slides[slide_index]['plots'] if plot['uid'] == plot_uid)
    plot['params'][param_name] = param_value
    update_plot_preview(slide_index, plot_uid)

# Initialize sidebar with thumbnails for each slide, each with a slide selection checkbox
def create_thumbnail(slide_index):
    thumbnail = widgets.Output()
    with thumbnail:
        fig, axs = plt.subplots(2, 2, figsize=(3, 3))
        position_map = {'top-left': (0, 0), 'top-right': (0, 1), 'bottom-left': (1, 0), 'bottom-right': (1, 1)}
        for plot in slides[slide_index]['plots']:
            pos = position_map[plot['position']]
            plot_graph(axs[pos], **plot['params'])
        plt.tight_layout()
        plt.close(fig)
        display(fig)

    # Add checkbox to select this slide
    checkbox = widgets.Checkbox(description=f"Select Slide {slide_index + 1}", value=False, layout=widgets.Layout(width='auto'))
    checkbox.observe(lambda change: show_slide(change, slide_index), names='value')
    slide_checkboxes.append(checkbox)
    return widgets.VBox([thumbnail, checkbox], layout=widgets.Layout(margin='5px'))

# Show slide when selected in the sidebar
def show_slide(change, slide_index):
    if change['new']:
        # Uncheck all other slide checkboxes to enforce single selection
        for cb in slide_checkboxes:
            if cb != change['owner']:
                cb.value = False
        render_slide(slide_index)

# Display main layout with sidebar and main slide display
def display_main_layout():
    global slide_checkboxes
    slide_checkboxes = []x  # Reset slide checkboxes
    sidebar.children = [create_thumbnail(i) for i in range(len(slides))]
    render_slide(selected_slide_index)  # Display the initial slide
    display(widgets.HBox([sidebar, main_slide_display, customization_window]))  # Ensure all sections are visible

# Initial display
display_main_layout()


In [76]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Slide and Plot Configurations
slides = [
    {'uid': 'slide_1', 'plots': [
        {'uid': 'plot_1', 'function_name': 'plot_graph', 'params': {'param1': 0.5, 'param2': 1.0, 'color': 'blue'}, 'position': 'top-left'},
        {'uid': 'plot_2', 'function_name': 'plot_graph', 'params': {'param1': 1.0, 'param2': 0.5, 'color': 'green'}, 'position': 'top-right'},
        {'uid': 'plot_3', 'function_name': 'plot_graph', 'params': {'param1': 0.8, 'param2': 0.8, 'color': 'red'}, 'position': 'bottom-left'},
        {'uid': 'plot_4', 'function_name': 'plot_graph', 'params': {'param1': 1.2, 'param2': 0.6, 'color': 'purple'}, 'position': 'bottom-right'}
    ]},
    {'uid': 'slide_2', 'plots': [
        {'uid': 'plot_5', 'function_name': 'plot_graph', 'params': {'param1': 0.3, 'param2': 1.2, 'color': 'orange'}, 'position': 'top-left'},
        {'uid': 'plot_6', 'function_name': 'plot_graph', 'params': {'param1': 1.1, 'param2': 0.7, 'color': 'cyan'}, 'position': 'top-right'},
        {'uid': 'plot_7', 'function_name': 'plot_graph', 'params': {'param1': 0.7, 'param2': 1.0, 'color': 'magenta'}, 'position': 'bottom-left'},
        {'uid': 'plot_8', 'function_name': 'plot_graph', 'params': {'param1': 1.3, 'param2': 0.9, 'color': 'yellow'}, 'position': 'bottom-right'}
    ]}
]

# Outputs for main slide display, sidebar, customization, and plot rendering
main_slide_display = widgets.Output(layout=widgets.Layout(width='60%', border='1px solid black'))
sidebar = widgets.VBox(layout=widgets.Layout(width='20%', overflow_y='auto', max_height='600px', border='1px solid black', padding='5px'))
customization_window = widgets.VBox(layout=widgets.Layout(width='20%', border='1px solid black'))
plot_output = widgets.Output()
selected_slide_index = 0
selected_plot_uid = None  # Tracks the current plot being customized
plot_checkboxes = []  # Track plot checkboxes globally to enforce single selection
slide_checkboxes = []  # Track slide checkboxes globally to enforce single selection

# Plotting function
def plot_graph(ax, **params):
    x = np.linspace(0, 10, 100)
    y = params['param1'] * np.sin(x) + params['param2'] * np.cos(x)
    ax.plot(x, y, color=params.get('color', 'blue'))
    ax.set_title("Customized Graph")
    ax.set_xlabel("X-axis")
    ax.set_ylabel("Y-axis")

# Function to render a slide with checkboxes at the top-right of each plot
def render_slide(slide_index):
    slide = slides[slide_index]
    global plot_checkboxes
    plot_checkboxes = []  # Reset plot_checkboxes for the current slide
    
    with main_slide_display:
        clear_output(wait=True)
        
        # Create grid layout for plots
        plot_grid = widgets.GridBox(
            children=[],
            layout=widgets.Layout(
                grid_template_columns="repeat(2, 1fr)",
                grid_gap="10px",
                align_items="center"
            )
        )

        # Display plots with checkboxes for selection within the same layout
        for plot in slide['plots']:
            plot_output = widgets.Output()
            checkbox = widgets.Checkbox(value=False, description=f"Select {plot['uid']}", indent=False, layout=widgets.Layout(width='auto'))
            checkbox.observe(lambda change, uid=plot['uid']: handle_plot_selection(change, slide_index, uid), names='value')
            plot_checkboxes.append(checkbox)

            with plot_output:
                fig, ax = plt.subplots(figsize=(5, 4))
                plot_graph(ax, **plot['params'])
                plt.show()

            # Arrange checkbox to appear above each plot on the top-right
            plot_box = widgets.VBox([
                widgets.HBox([checkbox], layout=widgets.Layout(justify_content='flex-end')),  # Checkbox aligned right
                plot_output
            ])
            plot_grid.children += (plot_box,)

        display(plot_grid)

# Function to handle plot selection for customization, ensuring only one checkbox is selected at a time
def handle_plot_selection(change, slide_index, plot_uid):
    global selected_plot_uid
    if change['new']:  # Only act if a plot checkbox is checked
        # Uncheck other checkboxes to enforce single selection
        for checkbox in plot_checkboxes:
            if checkbox.description != f"Select {plot_uid}":
                checkbox.value = False
        
        # Set the selected plot UID and open customization window
        selected_plot_uid = plot_uid
        open_customization_window(slide_index, plot_uid)

# Function to open the customization window for the selected plot
def open_customization_window(slide_index, plot_uid):
    customization_window.children = []  # Clear previous widgets
    slide = slides[slide_index]
    plot = next(plot for plot in slide['plots'] if plot['uid'] == plot_uid)
    params = plot['params']

    sliders = []
    for param_name, param_value in params.items():
        if param_name != 'color':  # Only add sliders for numeric params
            slider = widgets.FloatSlider(value=param_value, min=0.1, max=2.0, step=0.1, description=param_name)
            slider.observe(lambda change, name=param_name: update_plot_params(slide_index, plot_uid, name, change.new), names='value')
            sliders.append(slider)

    # Color picker widget
    color_picker = widgets.ColorPicker(
        value=params.get('color', 'blue'),
        description='Color',
        layout=widgets.Layout(width='auto')
    )
    color_picker.observe(lambda change: update_plot_params(slide_index, plot_uid, 'color', change.new), names='value')

    save_changes_checkbox = widgets.Checkbox(description="Save Changes", value=False)
    save_changes_checkbox.observe(lambda change: save_customization(slide_index, plot_uid), names='value')
    customization_window.children = [widgets.VBox(sliders + [color_picker, save_changes_checkbox]), plot_output]
    update_plot_preview(slide_index, plot_uid)

# Update plot preview in the customization window
def update_plot_preview(slide_index, plot_uid):
    plot = next(plot for plot in slides[slide_index]['plots'] if plot['uid'] == plot_uid)
    with plot_output:
        clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(5, 4))
        plot_graph(ax, **plot['params'])
        plt.show()

# Save customization and refresh main view and sidebar thumbnails
def save_customization(slide_index, plot_uid):
    render_slide(slide_index)  # Rerender the main slide view with updated parameters
    # Update the thumbnail for the current slide to reflect changes
    sidebar.children = [create_thumbnail(i) if i == slide_index else sidebar.children[i] for i in range(len(slides))]

# Update plot parameters in the slides list and refresh preview
def update_plot_params(slide_index, plot_uid, param_name, param_value):
    plot = next(plot for plot in slides[slide_index]['plots'] if plot['uid'] == plot_uid)
    plot['params'][param_name] = param_value
    update_plot_preview(slide_index, plot_uid)

# Initialize sidebar with thumbnails for each slide, each with a slide selection checkbox
def create_thumbnail(slide_index):
    thumbnail = widgets.Output()
    with thumbnail:
        fig, axs = plt.subplots(2, 2, figsize=(3, 3))
        position_map = {'top-left': (0, 0), 'top-right': (0, 1), 'bottom-left': (1, 0), 'bottom-right': (1, 1)}
        for plot in slides[slide_index]['plots']:
            pos = position_map[plot['position']]
            plot_graph(axs[pos], **plot['params'])
        plt.tight_layout()
        plt.close(fig)
        display(fig)

    # Add checkbox to select this slide
    checkbox = widgets.Checkbox(description=f"Select Slide {slide_index + 1}", value=False, layout=widgets.Layout(width='auto'))
    checkbox.observe(lambda change: show_slide(change, slide_index), names='value')
    slide_checkboxes.append(checkbox)
    return widgets.VBox([thumbnail, checkbox], layout=widgets.Layout(margin='5px'))

# Show slide when selected in the sidebar
def show_slide(change, slide_index):
    if change['new']:
        # Uncheck all other slide checkboxes to enforce single selection
        for cb in slide_checkboxes:
            if cb != change['owner']:
                cb.value = False
        render_slide(slide_index)

# Display main layout with sidebar and main slide display
def display_main_layout():
    global slide_checkboxes
    slide_checkboxes = []  # Reset slide checkboxes
    sidebar.children = [create_thumbnail(i) for i in range(len(slides))]
    render_slide(selected_slide_index)  # Display the initial slide
    display(widgets.VBox([main_slide_display, customization_window, sidebar]))  # Ensure all sections are visible

# Initial display
display_main_layout()


In [79]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Slide and Plot Configurations
slides = [
    {'uid': 'slide_1', 'plots': [
        {'uid': 'plot_1', 'function_name': 'plot_graph', 'params': {'param1': 0.5, 'param2': 1.0, 'color': 'blue'}, 'position': 'top-left'},
        {'uid': 'plot_2', 'function_name': 'plot_graph', 'params': {'param1': 1.0, 'param2': 0.5, 'color': 'green'}, 'position': 'top-right'},
        {'uid': 'plot_3', 'function_name': 'plot_graph', 'params': {'param1': 0.8, 'param2': 0.8, 'color': 'red'}, 'position': 'bottom-left'},
        {'uid': 'plot_4', 'function_name': 'plot_graph', 'params': {'param1': 1.2, 'param2': 0.6, 'color': 'purple'}, 'position': 'bottom-right'}
    ]},
    {'uid': 'slide_2', 'plots': [
        {'uid': 'plot_5', 'function_name': 'plot_graph', 'params': {'param1': 0.3, 'param2': 1.2, 'color': 'orange'}, 'position': 'top-left'},
        {'uid': 'plot_6', 'function_name': 'plot_graph', 'params': {'param1': 1.1, 'param2': 0.7, 'color': 'cyan'}, 'position': 'top-right'},
        {'uid': 'plot_7', 'function_name': 'plot_graph', 'params': {'param1': 0.7, 'param2': 1.0, 'color': 'magenta'}, 'position': 'bottom-left'},
        {'uid': 'plot_8', 'function_name': 'plot_graph', 'params': {'param1': 1.3, 'param2': 0.9, 'color': 'yellow'}, 'position': 'bottom-right'}
    ]}
]

# Outputs for main slide display, sidebar, customization, and plot rendering
main_slide_display = widgets.Output(layout=widgets.Layout(width='60%', border='1px solid black'))
sidebar = widgets.VBox(layout=widgets.Layout(width='20%', overflow_y='auto', max_height='600px', border='1px solid black', padding='5px'))
customization_window = widgets.VBox(layout=widgets.Layout(width='20%', border='1px solid black'))
plot_output = widgets.Output()
selected_slide_index = 0
selected_plot_uid = None  # Tracks the current plot being customized
plot_checkboxes = []  # Track plot checkboxes globally to enforce single selection
slide_checkboxes = []  # Track slide checkboxes globally to enforce single selection

# Plotting function
def plot_graph(ax, **params):
    x = np.linspace(0, 10, 100)
    y = params['param1'] * np.sin(x) + params['param2'] * np.cos(x)
    ax.plot(x, y, color=params.get('color', 'blue'))
    ax.set_title("Customized Graph")
    ax.set_xlabel("X-axis")
    ax.set_ylabel("Y-axis")

# Function to render a slide with checkboxes at the top-right of each plot
def render_slide(slide_index):
    slide = slides[slide_index]
    global plot_checkboxes
    plot_checkboxes = []  # Reset plot_checkboxes for the current slide
    
    with main_slide_display:
        clear_output(wait=True)
        
        # Create grid layout for plots
        plot_grid = widgets.GridBox(
            children=[],
            layout=widgets.Layout(
                grid_template_columns="repeat(2, 1fr)",
                grid_gap="10px",
                align_items="center"
            )
        )

        # Display plots with checkboxes for selection within the same layout
        for plot in slide['plots']:
            plot_output = widgets.Output()
            checkbox = widgets.Checkbox(value=False, description=f"Select {plot['uid']}", indent=False, layout=widgets.Layout(width='auto'))
            checkbox.observe(lambda change, uid=plot['uid']: handle_plot_selection(change, slide_index, uid), names='value')
            plot_checkboxes.append(checkbox)

            with plot_output:
                fig, ax = plt.subplots(figsize=(5, 4))
                plot_graph(ax, **plot['params'])
                plt.show()

            # Arrange checkbox to appear above each plot on the top-right
            plot_box = widgets.VBox([
                widgets.HBox([checkbox], layout=widgets.Layout(justify_content='flex-end')),  # Checkbox aligned right
                plot_output
            ])
            plot_grid.children += (plot_box,)

        display(plot_grid)

# Function to handle plot selection for customization, ensuring only one checkbox is selected at a time
def handle_plot_selection(change, slide_index, plot_uid):
    global selected_plot_uid
    if change['new']:  # Only act if a plot checkbox is checked
        # Uncheck other checkboxes to enforce single selection
        for checkbox in plot_checkboxes:
            if checkbox.description != f"Select {plot_uid}":
                checkbox.value = False
        
        # Set the selected plot UID and open customization window
        selected_plot_uid = plot_uid
        open_customization_window(slide_index, plot_uid)

# Function to open the customization window for the selected plot
def open_customization_window(slide_index, plot_uid):
    customization_window.children = []  # Clear previous widgets
    slide = slides[slide_index]
    plot = next(plot for plot in slide['plots'] if plot['uid'] == plot_uid)
    params = plot['params']

    sliders = []
    for param_name, param_value in params.items():
        if param_name != 'color':  # Only add sliders for numeric params
            slider = widgets.FloatSlider(value=param_value, min=0.1, max=2.0, step=0.1, description=param_name)
            slider.observe(lambda change, name=param_name: update_plot_params(slide_index, plot_uid, name, change.new), names='value')
            sliders.append(slider)

    # Color picker widget
    color_picker = widgets.ColorPicker(
        value=params.get('color', 'blue'),
        description='Color',
        layout=widgets.Layout(width='auto')
    )
    color_picker.observe(lambda change: update_plot_params(slide_index, plot_uid, 'color', change.new), names='value')

    save_changes_checkbox = widgets.Checkbox(description="Save Changes", value=False)
    save_changes_checkbox.observe(lambda change: save_customization(slide_index, plot_uid), names='value')
    customization_window.children = [widgets.VBox(sliders + [color_picker, save_changes_checkbox]), plot_output]
    update_plot_preview(slide_index, plot_uid)

# Update plot preview in the customization window
def update_plot_preview(slide_index, plot_uid):
    plot = next(plot for plot in slides[slide_index]['plots'] if plot['uid'] == plot_uid)
    with plot_output:
        clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(5, 4))
        plot_graph(ax, **plot['params'])
        plt.show()

# Save customization and refresh main view and sidebar thumbnails
def save_customization(slide_index, plot_uid):
    render_slide(slide_index)  # Rerender the main slide view with updated parameters
    # Update the thumbnail for the current slide to reflect changes
    sidebar.children[slide_index] = create_thumbnail(slide_index)

# Update plot parameters in the slides list and refresh preview
def update_plot_params(slide_index, plot_uid, param_name, param_value):
    plot = next(plot for plot in slides[slide_index]['plots'] if plot['uid'] == plot_uid)
    plot['params'][param_name] = param_value
    update_plot_preview(slide_index, plot_uid)

# Initialize sidebar with thumbnails for each slide, each with a slide selection checkbox
def create_thumbnail(slide_index):
    thumbnail = widgets.Output()
    with thumbnail:
        fig, axs = plt.subplots(2, 2, figsize=(3, 3))
        position_map = {'top-left': (0, 0), 'top-right': (0, 1), 'bottom-left': (1, 0), 'bottom-right': (1, 1)}
        for plot in slides[slide_index]['plots']:
            pos = position_map[plot['position']]
            plot_graph(axs[pos], **plot['params'])
        plt.tight_layout()
        plt.close(fig)
        display(fig)

    # Add checkbox to select this slide
    checkbox = widgets.Checkbox(description=f"Select Slide {slide_index + 1}", value=False, layout=widgets.Layout(width='auto'))
    checkbox.observe(lambda change: show_slide(change, slide_index), names='value')
    slide_checkboxes.append(checkbox)
    return widgets.VBox([thumbnail, checkbox], layout=widgets.Layout(margin='5px'))

# Show slide when selected in the sidebar
def show_slide(change, slide_index):
    if change['new']:
        # Uncheck all other slide checkboxes to enforce single selection
        for cb in slide_checkboxes:
            if cb != change['owner']:
                cb.value = False
        render_slide(slide_index)

# Display main layout with sidebar and main slide display
def display_main_layout():
    global slide_checkboxes
    slide_checkboxes = []  # Reset slide checkboxes
    sidebar.children = [create_thumbnail(i) for i in range(len(slides))]
    render_slide(selected_slide_index)  # Display the initial slide
    display(widgets.HBox([sidebar, main_slide_display, customization_window]))  # Ensure all sections are visible

# Initial display
display_main_layout()


In [80]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Slide and Plot Configurations
slides = [
    {'uid': 'slide_1', 'plots': [
        {'uid': 'plot_1', 'function_name': 'plot_graph', 'params': {'param1': 0.5, 'param2': 1.0, 'color': 'blue'}, 'position': 'top-left'},
        {'uid': 'plot_2', 'function_name': 'plot_graph', 'params': {'param1': 1.0, 'param2': 0.5, 'color': 'green'}, 'position': 'top-right'},
        {'uid': 'plot_3', 'function_name': 'plot_graph', 'params': {'param1': 0.8, 'param2': 0.8, 'color': 'red'}, 'position': 'bottom-left'},
        {'uid': 'plot_4', 'function_name': 'plot_graph', 'params': {'param1': 1.2, 'param2': 0.6, 'color': 'purple'}, 'position': 'bottom-right'}
    ]},
    {'uid': 'slide_2', 'plots': [
        {'uid': 'plot_5', 'function_name': 'plot_graph', 'params': {'param1': 0.3, 'param2': 1.2, 'color': 'orange'}, 'position': 'top-left'},
        {'uid': 'plot_6', 'function_name': 'plot_graph', 'params': {'param1': 1.1, 'param2': 0.7, 'color': 'cyan'}, 'position': 'top-right'},
        {'uid': 'plot_7', 'function_name': 'plot_graph', 'params': {'param1': 0.7, 'param2': 1.0, 'color': 'magenta'}, 'position': 'bottom-left'},
        {'uid': 'plot_8', 'function_name': 'plot_graph', 'params': {'param1': 1.3, 'param2': 0.9, 'color': 'yellow'}, 'position': 'bottom-right'}
    ]}
]

# Outputs for main slide display, sidebar, customization, and plot rendering
main_slide_display = widgets.Output(layout=widgets.Layout(width='60%', border='1px solid black'))
sidebar = widgets.VBox(layout=widgets.Layout(width='20%', overflow_y='auto', max_height='600px', border='1px solid black', padding='5px'))
customization_window = widgets.VBox(layout=widgets.Layout(width='20%', border='1px solid black'))
plot_output = widgets.Output()
selected_slide_index = 0
selected_plot_uid = None  # Tracks the current plot being customized
plot_checkboxes = []  # Track plot checkboxes globally to enforce single selection
slide_checkboxes = []  # Track slide checkboxes globally to enforce single selection

# Plotting function
def plot_graph(ax, **params):
    x = np.linspace(0, 10, 100)
    y = params['param1'] * np.sin(x) + params['param2'] * np.cos(x)
    ax.plot(x, y, color=params.get('color', 'blue'))
    ax.set_title("Customized Graph")
    ax.set_xlabel("X-axis")
    ax.set_ylabel("Y-axis")

# Function to render a slide with checkboxes at the top-right of each plot
def render_slide(slide_index):
    slide = slides[slide_index]
    global plot_checkboxes
    plot_checkboxes = []  # Reset plot_checkboxes for the current slide
    
    with main_slide_display:
        clear_output(wait=True)
        
        # Create grid layout for plots
        plot_grid = widgets.GridBox(
            children=[],
            layout=widgets.Layout(
                grid_template_columns="repeat(2, 1fr)",
                grid_gap="10px",
                align_items="center"
            )
        )

        # Display plots with checkboxes for selection within the same layout
        for plot in slide['plots']:
            plot_output = widgets.Output()
            checkbox = widgets.Checkbox(value=False, description=f"Select {plot['uid']}", indent=False, layout=widgets.Layout(width='auto'))
            checkbox.observe(lambda change, uid=plot['uid']: handle_plot_selection(change, slide_index, uid), names='value')
            plot_checkboxes.append(checkbox)

            with plot_output:
                fig, ax = plt.subplots(figsize=(5, 4))
                plot_graph(ax, **plot['params'])
                plt.show()

            # Arrange checkbox to appear above each plot on the top-right
            plot_box = widgets.VBox([
                widgets.HBox([checkbox], layout=widgets.Layout(justify_content='flex-end')),  # Checkbox aligned right
                plot_output
            ])
            plot_grid.children += (plot_box,)

        display(plot_grid)

# Function to handle plot selection for customization, ensuring only one checkbox is selected at a time
def handle_plot_selection(change, slide_index, plot_uid):
    global selected_plot_uid
    if change['new']:  # Only act if a plot checkbox is checked
        # Uncheck other checkboxes to enforce single selection
        for checkbox in plot_checkboxes:
            if checkbox.description != f"Select {plot_uid}":
                checkbox.value = False
        
        # Set the selected plot UID and open customization window
        selected_plot_uid = plot_uid
        open_customization_window(slide_index, plot_uid)

# Function to open the customization window for the selected plot
def open_customization_window(slide_index, plot_uid):
    customization_window.children = []  # Clear previous widgets
    slide = slides[slide_index]
    plot = next(plot for plot in slide['plots'] if plot['uid'] == plot_uid)
    params = plot['params']

    sliders = []
    for param_name, param_value in params.items():
        if param_name != 'color':  # Only add sliders for numeric params
            slider = widgets.FloatSlider(value=param_value, min=0.1, max=2.0, step=0.1, description=param_name)
            slider.observe(lambda change, name=param_name: update_plot_params(slide_index, plot_uid, name, change.new), names='value')
            sliders.append(slider)

    # Color picker widget
    color_picker = widgets.ColorPicker(
        value=params.get('color', 'blue'),
        description='Color',
        layout=widgets.Layout(width='auto')
    )
    color_picker.observe(lambda change: update_plot_params(slide_index, plot_uid, 'color', change.new), names='value')

    save_changes_checkbox = widgets.Checkbox(description="Save Changes", value=False)
    save_changes_checkbox.observe(lambda change: save_customization(slide_index, plot_uid), names='value')
    customization_window.children = [widgets.VBox(sliders + [color_picker, save_changes_checkbox]), plot_output]
    update_plot_preview(slide_index, plot_uid)

# Update plot preview in the customization window
def update_plot_preview(slide_index, plot_uid):
    plot = next(plot for plot in slides[slide_index]['plots'] if plot['uid'] == plot_uid)
    with plot_output:
        clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(5, 4))
        plot_graph(ax, **plot['params'])
        plt.show()

# Save customization and refresh main view and sidebar thumbnails
def save_customization(slide_index, plot_uid):
    render_slide(slide_index)  # Rerender the main slide view with updated parameters
    # Update the thumbnail for the current slide to reflect changes
    sidebar.children = tuple(
        create_thumbnail(i) if i == slide_index else sidebar.children[i] for i in range(len(slides))
    )

# Update plot parameters in the slides list and refresh preview
def update_plot_params(slide_index, plot_uid, param_name, param_value):
    plot = next(plot for plot in slides[slide_index]['plots'] if plot['uid'] == plot_uid)
    plot['params'][param_name] = param_value
    update_plot_preview(slide_index, plot_uid)

# Initialize sidebar with thumbnails for each slide, each with a slide selection checkbox
def create_thumbnail(slide_index):
    thumbnail = widgets.Output()
    with thumbnail:
        fig, axs = plt.subplots(2, 2, figsize=(3, 3))
        position_map = {'top-left': (0, 0), 'top-right': (0, 1), 'bottom-left': (1, 0), 'bottom-right': (1, 1)}
        for plot in slides[slide_index]['plots']:
            pos = position_map[plot['position']]
            plot_graph(axs[pos], **plot['params'])
        plt.tight_layout()
        plt.close(fig)
        display(fig)

    # Add checkbox to select this slide
    checkbox = widgets.Checkbox(description=f"Select Slide {slide_index + 1}", value=False, layout=widgets.Layout(width='auto'))
    checkbox.observe(lambda change: show_slide(change, slide_index), names='value')
    slide_checkboxes.append(checkbox)
    return widgets.VBox([thumbnail, checkbox], layout=widgets.Layout(margin='5px'))

# Show slide when selected in the sidebar
def show_slide(change, slide_index):
    if change['new']:
        # Uncheck all other slide checkboxes to enforce single selection
        for cb in slide_checkboxes:
            if cb != change['owner']:
                cb.value = False
        render_slide(slide_index)

# Display main layout with sidebar and main slide display
def display_main_layout():
    global slide_checkboxes
    slide_checkboxes = []  # Reset slide checkboxes
    sidebar.children = [create_thumbnail(i) for i in range(len(slides))]
    render_slide(selected_slide_index)  # Display the initial slide
    display(widgets.HBox([sidebar, main_slide_display, customization_window]))  # Ensure all sections are visible

# Initial display
display_main_layout()

In [90]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from pptx import Presentation
from pptx.util import Inches

# Directory for saving PNG files
output_dir = 'exported_images'
os.makedirs(output_dir, exist_ok=True)

# Slide and Plot Configurations
slides = [
    {'uid': 'slide_1', 'plots': [
        {'uid': 'plot_1', 'function_name': 'plot_graph', 'params': {'param1': 0.5, 'param2': 1.0, 'color': 'blue'}, 'position': 'top-left'},
        {'uid': 'plot_2', 'function_name': 'plot_graph', 'params': {'param1': 1.0, 'param2': 0.5, 'color': 'green'}, 'position': 'top-right'},
        {'uid': 'plot_3', 'function_name': 'plot_graph', 'params': {'param1': 0.8, 'param2': 0.8, 'color': 'red'}, 'position': 'bottom-left'},
        {'uid': 'plot_4', 'function_name': 'plot_graph', 'params': {'param1': 1.2, 'param2': 0.6, 'color': 'purple'}, 'position': 'bottom-right'}
    ]}
]

# Outputs for main slide display, sidebar, customization, and plot rendering
main_slide_display = widgets.Output(layout=widgets.Layout(width='60%', border='1px solid black'))
sidebar = widgets.VBox(layout=widgets.Layout(width='20%', overflow_y='auto', max_height='600px', border='1px solid black', padding='5px'))
customization_window = widgets.VBox(layout=widgets.Layout(width='20%', border='1px solid black'))
plot_output = widgets.Output()
selected_slide_index = 0
selected_plot_uid = None  # Tracks the current plot being customized
plot_checkboxes = []  # Track plot checkboxes globally to enforce single selection
slide_checkboxes = []  # Track slide checkboxes globally to enforce single selection

# Plotting function
def plot_graph(ax, **params):
    x = np.linspace(0, 10, 100)
    y = params['param1'] * np.sin(x) + params['param2'] * np.cos(x)
    ax.plot(x, y, color=params.get('color', 'blue'))
    ax.set_title("Customized Graph")
    ax.set_xlabel("X-axis")
    ax.set_ylabel("Y-axis")

# Function to generate PNG from each plot
def run_plot_functions(df, function_registry):
    for index, row in df.iterrows():
        func_name = row['function_name']
        func = function_registry.get(func_name)
        if func:
            output_path = os.path.join(output_dir, f"{func_name}_{index}.png")
            fig, ax = plt.subplots()
            func(ax, **row['input_values'])
            fig.savefig(output_path)
            plt.close(fig)
            print(f"Generated PNG at {output_path}")
        else:
            print(f"Function '{func_name}' not found in registry")

# Function to add images to PPT
def build_ppt(prs, df):
    for index, row in df.iterrows():
        slide_num, position = row['layout']
        title = row['graph_title']
        source = row['source']
        func_name = row['function_name']
        
        img_path = os.path.join(output_dir, f"{func_name}_{index}.png")
        
        if not os.path.exists(img_path):
            print(f"Image {img_path} not found")
            continue
        
        if slide_num >= len(prs.slides):
            slide_layout = prs.slide_layouts[5]
            prs.slides.add_slide(slide_layout)
        
        slide = prs.slides[slide_num]
        title_placeholder = slide.shapes.title
        title_placeholder.text = title
        
        left = Inches(1) if position == 'left' else Inches(5)
        top = Inches(1) if position == 'top' else Inches(4)
        
        slide.shapes.add_picture(img_path, left, top, width=Inches(4))
        
        print(f"Added {img_path} to slide {slide_num} at {position}")

# Wrapper for Export button
def on_export_button_click(button):
    print("Starting export process...")
    
    # Mock DataFrame for function execution
    df = pd.DataFrame([
        {'function_name': 'plot_graph', 'input_values': {'param1': 1, 'param2': 1, 'color': 'blue'}, 'layout': (0, 'left'), 'graph_title': 'Example Plot', 'source': 'Source Info'}
    ])
    
    function_registry = {'plot_graph': plot_graph}
    
    # Step 1: Generate PNGs
    run_plot_functions(df, function_registry)
    
    # Step 2: Populate PPT with PNGs
    prs = Presentation()
    build_ppt(prs, df)
    ppt_path = 'output_presentation.pptx'
    prs.save(ppt_path)
    print(f"Presentation saved as {ppt_path}")

# Add Export button to UI
export_button = widgets.Button(description="Export to PPT")
export_button.on_click(on_export_button_click)

# Display UI
display(widgets.VBox([main_slide_display, customization_window, export_button]))


In [91]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from pptx import Presentation
from pptx.util import Inches

# Directory for saving PNG files
output_dir = 'exported_images'
os.makedirs(output_dir, exist_ok=True)

# Slide and Plot Configurations
slides = [
    {'uid': 'slide_1', 'plots': [
        {'uid': 'plot_1', 'function_name': 'plot_graph', 'params': {'param1': 0.5, 'param2': 1.0, 'color': 'blue'}, 'position': 'top-left'},
        {'uid': 'plot_2', 'function_name': 'plot_graph', 'params': {'param1': 1.0, 'param2': 0.5, 'color': 'green'}, 'position': 'top-right'},
        {'uid': 'plot_3', 'function_name': 'plot_graph', 'params': {'param1': 0.8, 'param2': 0.8, 'color': 'red'}, 'position': 'bottom-left'},
        {'uid': 'plot_4', 'function_name': 'plot_graph', 'params': {'param1': 1.2, 'param2': 0.6, 'color': 'purple'}, 'position': 'bottom-right'}
    ]},
    {'uid': 'slide_2', 'plots': [
        {'uid': 'plot_5', 'function_name': 'plot_graph', 'params': {'param1': 0.3, 'param2': 1.2, 'color': 'orange'}, 'position': 'top-left'},
        {'uid': 'plot_6', 'function_name': 'plot_graph', 'params': {'param1': 1.1, 'param2': 0.7, 'color': 'cyan'}, 'position': 'top-right'},
        {'uid': 'plot_7', 'function_name': 'plot_graph', 'params': {'param1': 0.7, 'param2': 1.0, 'color': 'magenta'}, 'position': 'bottom-left'},
        {'uid': 'plot_8', 'function_name': 'plot_graph', 'params': {'param1': 1.3, 'param2': 0.9, 'color': 'yellow'}, 'position': 'bottom-right'}
    ]}
]

# Outputs for main slide display, sidebar, customization, and plot rendering
main_slide_display = widgets.Output(layout=widgets.Layout(width='60%', border='1px solid black'))
sidebar = widgets.VBox(layout=widgets.Layout(width='20%', overflow_y='auto', max_height='600px', border='1px solid black', padding='5px'))
customization_window = widgets.VBox(layout=widgets.Layout(width='20%', border='1px solid black'))
plot_output = widgets.Output()
selected_slide_index = 0
selected_plot_uid = None  # Tracks the current plot being customized
plot_checkboxes = []  # Track plot checkboxes globally to enforce single selection
slide_checkboxes = []  # Track slide checkboxes globally to enforce single selection

# Plotting function
def plot_graph(ax, **params):
    x = np.linspace(0, 10, 100)
    y = params['param1'] * np.sin(x) + params['param2'] * np.cos(x)
    ax.plot(x, y, color=params.get('color', 'blue'))
    ax.set_title("Customized Graph")
    ax.set_xlabel("X-axis")
    ax.set_ylabel("Y-axis")

# Function to generate PNG from each plot
def run_plot_functions(df, function_registry):
    for index, row in df.iterrows():
        func_name = row['function_name']
        func = function_registry.get(func_name)
        if func:
            output_path = os.path.join(output_dir, f"{func_name}_{index}.png")
            fig, ax = plt.subplots()
            func(ax, **row['input_values'])
            fig.savefig(output_path)
            plt.close(fig)
            print(f"Generated PNG at {output_path}")
        else:
            print(f"Function '{func_name}' not found in registry")

# Function to add images to PPT
def build_ppt(prs, df):
    for index, row in df.iterrows():
        slide_num, position = row['layout']
        title = row['graph_title']
        source = row['source']
        func_name = row['function_name']
        
        img_path = os.path.join(output_dir, f"{func_name}_{index}.png")
        
        if not os.path.exists(img_path):
            print(f"Image {img_path} not found")
            continue
        
        if slide_num >= len(prs.slides):
            slide_layout = prs.slide_layouts[5]
            prs.slides.add_slide(slide_layout)
        
        slide = prs.slides[slide_num]
        title_placeholder = slide.shapes.title
        title_placeholder.text = title
        
        left = Inches(1) if position == 'left' else Inches(5)
        top = Inches(1) if position == 'top' else Inches(4)
        
        slide.shapes.add_picture(img_path, left, top, width=Inches(4))
        
        print(f"Added {img_path} to slide {slide_num} at {position}")

# Wrapper for Export button
def on_export_button_click(button):
    print("Starting export process...")
    
    # Prepare DataFrame for function execution based on UI selections
    data = []
    for slide in slides:
        for plot in slide['plots']:
            data.append({
                'function_name': plot['function_name'],
                'input_values': plot['params'],
                'layout': (slides.index(slide), plot['position']),
                'graph_title': f"Plot {plot['uid']}",
                'source': "Generated by App"
            })
    
    df = pd.DataFrame(data)
    
    function_registry = {'plot_graph': plot_graph}
    
    # Step 1: Generate PNGs
    run_plot_functions(df, function_registry)
    
    # Step 2: Populate PPT with PNGs
    prs = Presentation()
    build_ppt(prs, df)
    ppt_path = 'output_presentation.pptx'
    prs.save(ppt_path)
    print(f"Presentation saved as {ppt_path}")

# Add Export button to UI
export_button = widgets.Button(description="Export to PPT")
export_button.on_click(on_export_button_click)

# Display UI components
def display_main_layout():
    global slide_checkboxes
    slide_checkboxes = []  # Reset slide checkboxes
    sidebar.children = [create_thumbnail(i) for i in range(len(slides))]
    render_slide(selected_slide_index)  # Display the initial slide
    display(widgets.VBox([widgets.HBox([sidebar, main_slide_display, customization_window]), export_button]))  # Ensure all sections are visible

# Initial display
display_main_layout()


In [92]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from pptx import Presentation
from pptx.util import Inches

# Directory for saving PNG files
output_dir = 'exported_images'
os.makedirs(output_dir, exist_ok=True)

# Slide and Plot Configurations
slides = [
    {'uid': 'slide_1', 'plots': [
        {'uid': 'plot_1', 'function_name': 'plot_graph', 'params': {'param1': 0.5, 'param2': 1.0, 'color': 'blue'}, 'position': 'top-left'},
        {'uid': 'plot_2', 'function_name': 'plot_graph', 'params': {'param1': 1.0, 'param2': 0.5, 'color': 'green'}, 'position': 'top-right'},
        {'uid': 'plot_3', 'function_name': 'plot_graph', 'params': {'param1': 0.8, 'param2': 0.8, 'color': 'red'}, 'position': 'bottom-left'},
        {'uid': 'plot_4', 'function_name': 'plot_graph', 'params': {'param1': 1.2, 'param2': 0.6, 'color': 'purple'}, 'position': 'bottom-right'}
    ]},
    {'uid': 'slide_2', 'plots': [
        {'uid': 'plot_5', 'function_name': 'plot_graph', 'params': {'param1': 0.3, 'param2': 1.2, 'color': 'orange'}, 'position': 'top-left'},
        {'uid': 'plot_6', 'function_name': 'plot_graph', 'params': {'param1': 1.1, 'param2': 0.7, 'color': 'cyan'}, 'position': 'top-right'},
        {'uid': 'plot_7', 'function_name': 'plot_graph', 'params': {'param1': 0.7, 'param2': 1.0, 'color': 'magenta'}, 'position': 'bottom-left'},
        {'uid': 'plot_8', 'function_name': 'plot_graph', 'params': {'param1': 1.3, 'param2': 0.9, 'color': 'yellow'}, 'position': 'bottom-right'}
    ]}
]

# Outputs for main slide display, sidebar, customization, and plot rendering
main_slide_display = widgets.Output(layout=widgets.Layout(width='60%', border='1px solid black'))
sidebar = widgets.VBox(layout=widgets.Layout(width='20%', overflow_y='auto', max_height='600px', border='1px solid black', padding='5px'))
customization_window = widgets.VBox(layout=widgets.Layout(width='20%', border='1px solid black'))
plot_output = widgets.Output()
selected_slide_index = 0
selected_plot_uid = None  # Tracks the current plot being customized
plot_checkboxes = []  # Track plot checkboxes globally to enforce single selection
slide_checkboxes = []  # Track slide checkboxes globally to enforce single selection

# Plotting function
def plot_graph(ax, **params):
    x = np.linspace(0, 10, 100)
    y = params['param1'] * np.sin(x) + params['param2'] * np.cos(x)
    ax.plot(x, y, color=params.get('color', 'blue'))
    ax.set_title("Customized Graph")
    ax.set_xlabel("X-axis")
    ax.set_ylabel("Y-axis")

# Function to generate PNG from each plot
def run_plot_functions(df, function_registry):
    for index, row in df.iterrows():
        func_name = row['function_name']
        func = function_registry.get(func_name)
        if func:
            output_path = os.path.join(output_dir, f"{func_name}_{index}.png")
            fig, ax = plt.subplots()
            func(ax, **row['input_values'])
            fig.savefig(output_path)
            plt.close(fig)
            print(f"Generated PNG at {output_path}")
        else:
            print(f"Function '{func_name}' not found in registry")

# Function to add images to PPT
def build_ppt(prs, df):
    for index, row in df.iterrows():
        slide_num, position = row['layout']
        title = row['graph_title']
        source = row['source']
        func_name = row['function_name']
        
        img_path = os.path.join(output_dir, f"{func_name}_{index}.png")
        
        if not os.path.exists(img_path):
            print(f"Image {img_path} not found")
            continue
        
        if slide_num >= len(prs.slides):
            slide_layout = prs.slide_layouts[5]
            prs.slides.add_slide(slide_layout)
        
        slide = prs.slides[slide_num]
        title_placeholder = slide.shapes.title
        title_placeholder.text = title
        
        left = Inches(1) if position == 'left' else Inches(5)
        top = Inches(1) if position == 'top' else Inches(4)
        
        slide.shapes.add_picture(img_path, left, top, width=Inches(4))
        
        print(f"Added {img_path} to slide {slide_num} at {position}")

# Wrapper for Export button
def on_export_button_click(button):
    print("Starting export process...")
    
    # Prepare DataFrame for function execution based on UI selections
    data = []
    for slide in slides:
        for plot in slide['plots']:
            data.append({
                'function_name': plot['function_name'],
                'input_values': plot['params'],
                'layout': (slides.index(slide), plot['position']),
                'graph_title': f"Plot {plot['uid']}",
                'source': "Generated by App"
            })
    
    df = pd.DataFrame(data)
    
    function_registry = {'plot_graph': plot_graph}
    
    # Step 1: Generate PNGs
    run_plot_functions(df, function_registry)
    
    # Step 2: Populate PPT with PNGs
    prs = Presentation()
    build_ppt(prs, df)
    ppt_path = 'output_presentation.pptx'
    prs.save(ppt_path)
    print(f"Presentation saved as {ppt_path}")

# Add Export button to UI
export_button = widgets.Button(description="Export to PPT")
export_button.on_click(on_export_button_click)

# Display UI components
def display_main_layout():
    global slide_checkboxes
    slide_checkboxes = []  # Reset slide checkboxes
    sidebar.children = [create_thumbnail(i) for i in range(len(slides))]
    render_slide(selected_slide_index)  # Display the initial slide
    display(widgets.VBox([widgets.HBox([sidebar, main_slide_display, customization_window]), export_button]))  # Ensure all sections are visible

# Function to update main view and thumbnail on save
def save_customization(slide_index, plot_uid):
    render_slide(slide_index)  # Rerender the main slide view with updated parameters
    sidebar.children = tuple(
        create_thumbnail(i) if i == slide_index else sidebar.children[i] for i in range(len(slides))
    )

# Function to handle plot selection for customization, ensuring only one checkbox is selected at a time
def handle_plot_selection(change, slide_index, plot_uid):
    global selected_plot_uid
    if change['new']:  # Only act if a plot checkbox is checked
        # Uncheck other checkboxes to enforce single selection
        for checkbox in plot_checkboxes:
            if checkbox.description != f"Select {plot_uid}":
                checkbox.value = False
        
        # Set the selected plot UID and open customization window
        selected_plot_uid = plot_uid
        open_customization_window(slide_index, plot_uid)

# Function to open the customization window for the selected plot
def open_customization_window(slide_index, plot_uid):
    customization_window.children = []  # Clear previous widgets
    slide = slides[slide_index]
    plot = next(plot for plot in slide['plots'] if plot['uid'] == plot_uid)
    params = plot['params']

    sliders = []
    for param_name, param_value in params.items():
        if param_name != 'color':  # Only add sliders for numeric params
            slider = widgets.FloatSlider(value=param_value, min=0.1, max=2.0, step=0.1, description=param_name)
            slider.observe(lambda change, name=param_name: update_plot_params(slide_index, plot_uid, name, change.new), names='value')
            sliders.append(slider)

    # Color picker widget
    color_picker = widgets.ColorPicker(
        value=params.get('color', 'blue'),
        description='Color',
        layout=widgets.Layout(width='auto')
    )
    color_picker.observe(lambda change: update_plot_params(slide_index, plot_uid, 'color', change.new), names='value')

    save_changes_checkbox = widgets.Checkbox(description="Save Changes", value=False)
    save_changes_checkbox.observe(lambda change: save_customization(slide_index, plot_uid), names='value')
    customization_window.children = [widgets.VBox(sliders + [color_picker, save_changes_checkbox]), plot_output]
    update_plot_preview(slide_index, plot_uid)

# Update plot preview in the customization window
def update_plot_preview(slide_index, plot_uid):
    plot = next(plot for plot in slides[slide_index]['plots'] if plot['uid'] == plot_uid)
    with plot_output:
        clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(5, 4))
        plot_graph(ax, **plot['params'])
        plt.show()

# Update plot parameters in the slides list and refresh preview
def update_plot_params(slide_index, plot_uid, param_name, param_value):
    plot = next(plot for plot in slides[slide_index]['plots'] if plot['uid'] == plot_uid)
    plot['params'][param_name] = param_value
    update_plot_preview(slide_index, plot_uid)

# Initialize sidebar with thumbnails for each slide, each with a slide selection checkbox
def create_thumbnail(slide_index):
    thumbnail = widgets.Output()
    with thumbnail:
        fig, axs = plt.subplots(2, 2, figsize=(3, 3))
        position_map = {'top-left': (0, 0), 'top-right': (0, 1), 'bottom-left': (1, 0), 'bottom-right': (1, 1)}
        for plot in slides[slide_index]['plots']:
            pos = position_map[plot['position']]
            plot_graph(axs[pos], **plot['params'])
        plt.tight_layout()
        plt.close(fig)
        display(fig)

    # Add checkbox to select this slide
    checkbox = widgets.Checkbox(description=f"Select Slide {slide_index + 1}", value=False, layout=widgets.Layout(width='auto'))
    checkbox.observe(lambda change: show_slide(change, slide_index), names='value')
    slide_checkboxes.append(checkbox)
    return widgets.VBox([thumbnail, checkbox], layout=widgets.Layout(margin='5px'))

# Show slide when selected in the sidebar
def show_slide(change, slide_index):
    if change['new']:
        # Uncheck all other slide checkboxes to enforce single selection
        for cb in slide_checkboxes:
            if cb != change['owner']:
                cb.value = False
        render_slide(slide_index)

# Initial display
display_main_layout()


### Clean 

In [107]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from pptx import Presentation
from pptx.util import Inches

# Directory for saving PNG files
output_dir = 'exported_images'
os.makedirs(output_dir, exist_ok=True)

# Slide and Plot Configurations
slides = [
    {'uid': 'slide_1', 'plots': [
        {'uid': 'plot_1', 'function_name': 'plot_graph', 'params': {'param1': 0.5, 'param2': 1.0, 'color': 'blue'}, 'position': 'top-left'},
        {'uid': 'plot_2', 'function_name': 'plot_graph', 'params': {'param1': 1.0, 'param2': 0.5, 'color': 'green'}, 'position': 'top-right'},
        {'uid': 'plot_3', 'function_name': 'plot_graph', 'params': {'param1': 0.8, 'param2': 0.8, 'color': 'red'}, 'position': 'bottom-left'},
        {'uid': 'plot_4', 'function_name': 'plot_graph', 'params': {'param1': 1.2, 'param2': 0.6, 'color': 'purple'}, 'position': 'bottom-right'}
    ]},
    {'uid': 'slide_2', 'plots': [
        {'uid': 'plot_5', 'function_name': 'plot_graph', 'params': {'param1': 0.3, 'param2': 1.2, 'color': 'orange'}, 'position': 'top-left'},
        {'uid': 'plot_6', 'function_name': 'plot_graph', 'params': {'param1': 1.1, 'param2': 0.7, 'color': 'cyan'}, 'position': 'top-right'},
        {'uid': 'plot_7', 'function_name': 'plot_graph', 'params': {'param1': 0.7, 'param2': 1.0, 'color': 'magenta'}, 'position': 'bottom-left'},
        {'uid': 'plot_8', 'function_name': 'plot_graph', 'params': {'param1': 1.3, 'param2': 0.9, 'color': 'yellow'}, 'position': 'bottom-right'}
    ]}
]

# Outputs for main slide display, sidebar, customization, and plot rendering
main_slide_display = widgets.Output(layout=widgets.Layout(width='60%', border='1px solid black'))
sidebar = widgets.VBox(layout=widgets.Layout(width='20%', overflow_y='auto', max_height='600px', border='1px solid black', padding='5px'))
customization_window = widgets.VBox(layout=widgets.Layout(width='20%', border='1px solid black'))
plot_output = widgets.Output()
selected_slide_index = 0
selected_plot_uid = None
plot_checkboxes = []
slide_checkboxes = []

# Plotting function
def plot_graph(ax, **params):
    x = np.linspace(0, 10, 100)
    y = params['param1'] * np.sin(x) + params['param2'] * np.cos(x)
    ax.plot(x, y, color=params.get('color', 'blue'))
    ax.set_title("Customized Graph")
    ax.set_xlabel("X-axis")
    ax.set_ylabel("Y-axis")

# Function to generate PNGs from each plot
def run_plot_functions(df, function_registry):
    for index, row in df.iterrows():
        func_name = row['function_name']
        func = function_registry.get(func_name)
        if func:
            output_path = os.path.join(output_dir, f"{row['plot_uid']}.png")
            fig, ax = plt.subplots()
            func(ax, **row['input_values'])
            fig.savefig(output_path)
            plt.close(fig)
            print(f"Generated PNG at {output_path}")
        else:
            print(f"Function '{func_name}' not found in registry")

# Function to add images to PPT with layout consistency
def build_ppt(prs, df):
    for slide_index in df['slide_index'].unique():
        slide_df = df[df['slide_index'] == slide_index]
        
        # Create a new slide
        slide_layout = prs.slide_layouts[5]
        slide = prs.slides.add_slide(slide_layout)

        for _, row in slide_df.iterrows():
            title = row['graph_title']
            img_path = os.path.join(output_dir, f"{row['plot_uid']}.png")
            
            if not os.path.exists(img_path):
                print(f"Image {img_path} not found")
                continue
            
            # Set positions based on plot position in a 2x2 grid
            if row['position'] == 'top-left':
                left, top = Inches(1), Inches(1)
            elif row['position'] == 'top-right':
                left, top = Inches(5), Inches(1)
            elif row['position'] == 'bottom-left':
                left, top = Inches(1), Inches(4)
            else:  # 'bottom-right'
                left, top = Inches(5), Inches(4)

            # Add title and image
            slide.shapes.add_picture(img_path, left, top, width=Inches(4))
            title_placeholder = slide.shapes.title
            title_placeholder.text = f"Slide {slide_index + 1}: {title}"

# Export button function
def on_export_button_click(button):
    print("Starting export process...")
    
    data = []
    for slide in slides:
        for plot in slide['plots']:
            data.append({
                'slide_index': slides.index(slide),
                'plot_uid': plot['uid'],
                'function_name': plot['function_name'],
                'input_values': plot['params'],
                'position': plot['position'],
                'graph_title': f"Plot {plot['uid']}"
            })
    
    df = pd.DataFrame(data)
    function_registry = {'plot_graph': plot_graph}
    
    # Generate PNGs and create PPT
    run_plot_functions(df, function_registry)
    prs = Presentation()
    build_ppt(prs, df)
    prs.save('output_presentation.pptx')
    print("Presentation saved as output_presentation.pptx")

# Add Export button to UI
export_button = widgets.Button(description="Export to PPT")
export_button.on_click(on_export_button_click)

# Display the main layout with Export button
def display_main_layout():
    global slide_checkboxes
    slide_checkboxes = []
    sidebar.children = [create_thumbnail(i) for i in range(len(slides))]
    render_slide(selected_slide_index)
    display(widgets.VBox([widgets.HBox([sidebar, main_slide_display, customization_window]), export_button]))

# Function to save changes to plots and update main view and thumbnail
def save_customization(slide_index, plot_uid):
    render_slide(slide_index)
    sidebar.children = tuple(
        create_thumbnail(i) if i == slide_index else sidebar.children[i] for i in range(len(slides))
    )

# Display and update functionalities remain as before...
# Initialize UI
display_main_layout()

### LAST - Inconsistency between icon, dropdown menu and 

In [113]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Slide and Plot Configurations
slides = [
    {'uid': 'slide_1', 'plots': [
        {'uid': 'plot_1', 'function_name': 'plot_graph', 'params': {'param1': 0.5, 'param2': 1.0, 'color': 'blue'}, 'position': 'top-left'},
        {'uid': 'plot_2', 'function_name': 'plot_graph', 'params': {'param1': 1.0, 'param2': 0.5, 'color': 'green'}, 'position': 'top-right'},
    ]},
    {'uid': 'slide_2', 'plots': [
        {'uid': 'plot_3', 'function_name': 'plot_graph', 'params': {'param1': 0.8, 'param2': 0.8, 'color': 'red'}, 'position': 'bottom-left'},
    ]}
]

# Outputs for main slide display, sidebar, customization, and plot rendering
main_slide_display = widgets.Output(layout=widgets.Layout(width='60%', border='1px solid black'))
sidebar = widgets.VBox(layout=widgets.Layout(width='20%', overflow_y='auto', max_height='600px', border='1px solid black', padding='5px'))
customization_window = widgets.VBox(layout=widgets.Layout(width='20%', border='1px solid black'))
plot_output = widgets.Output()
selected_slide_index = None  # No slide selected initially
plot_checkboxes = []  # Track plot checkboxes globally to enforce single selection
slide_checkboxes = []  # Track slide checkboxes globally to enforce single selection

# Plotting function
def plot_graph(ax, **params):
    x = np.linspace(0, 10, 100)
    y = params['param1'] * np.sin(x) + params['param2'] * np.cos(x)
    ax.plot(x, y, color=params.get('color', 'blue'))
    ax.set_title("Customized Graph")
    ax.set_xlabel("X-axis")
    ax.set_ylabel("Y-axis")

# Function to render a slide with checkboxes at the top-right of each plot
def render_slide(slide_index):
    slide = slides[slide_index]
    global plot_checkboxes
    plot_checkboxes = []  # Reset plot_checkboxes for the current slide
    
    with main_slide_display:
        clear_output(wait=True)
        
        # Create grid layout for plots
        plot_grid = widgets.GridBox(
            children=[],
            layout=widgets.Layout(
                grid_template_columns="repeat(2, 1fr)",
                grid_gap="10px",
                align_items="center"
            )
        )

        # Display plots with checkboxes for selection within the same layout
        for plot in slide['plots']:
            plot_output = widgets.Output()
            checkbox = widgets.Checkbox(value=False, description=f"Select {plot['uid']}", indent=False, layout=widgets.Layout(width='auto'))
            checkbox.observe(lambda change, uid=plot['uid']: handle_plot_selection(change, slide_index, uid), names='value')
            plot_checkboxes.append(checkbox)

            with plot_output:
                fig, ax = plt.subplots(figsize=(5, 4))
                plot_graph(ax, **plot['params'])
                plt.show()

            # Arrange checkbox to appear above each plot on the top-right
            plot_box = widgets.VBox([
                widgets.HBox([checkbox], layout=widgets.Layout(justify_content='flex-end')),  # Checkbox aligned right
                plot_output
            ])
            plot_grid.children += (plot_box,)

        display(plot_grid)

# Function to handle plot selection for customization, ensuring only one checkbox is selected at a time
def handle_plot_selection(change, slide_index, plot_uid):
    global selected_plot_uid
    if change['new']:  # Only act if a plot checkbox is checked
        # Uncheck other checkboxes to enforce single selection
        for checkbox in plot_checkboxes:
            if checkbox.description != f"Select {plot_uid}":
                checkbox.value = False
        
        # Set the selected plot UID and open customization window
        selected_plot_uid = plot_uid
        open_customization_window(slide_index, plot_uid)

# Function to open the customization window for the selected plot
def open_customization_window(slide_index, plot_uid):
    customization_window.children = []  # Clear previous widgets
    slide = slides[slide_index]
    plot = next(plot for plot in slide['plots'] if plot['uid'] == plot_uid)
    params = plot['params']

    sliders = []
    for param_name, param_value in params.items():
        if param_name != 'color':  # Only add sliders for numeric params
            slider = widgets.FloatSlider(value=param_value, min=0.1, max=2.0, step=0.1, description=param_name)
            slider.observe(lambda change, name=param_name: update_plot_params(slide_index, plot_uid, name, change.new), names='value')
            sliders.append(slider)

    # Color picker widget
    color_picker = widgets.ColorPicker(
        value=params.get('color', 'blue'),
        description='Color',
        layout=widgets.Layout(width='auto')
    )
    color_picker.observe(lambda change: update_plot_params(slide_index, plot_uid, 'color', change.new), names='value')

    save_changes_checkbox = widgets.Checkbox(description="Save Changes", value=False)
    save_changes_checkbox.observe(lambda change: save_customization(slide_index, plot_uid), names='value')
    customization_window.children = [widgets.VBox(sliders + [color_picker, save_changes_checkbox]), plot_output]
    update_plot_preview(slide_index, plot_uid)

# Update plot preview in the customization window
def update_plot_preview(slide_index, plot_uid):
    plot = next(plot for plot in slides[slide_index]['plots'] if plot['uid'] == plot_uid)
    with plot_output:
        clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(5, 4))
        plot_graph(ax, **plot['params'])
        plt.show()

# Save customization and refresh main view and sidebar thumbnails
def save_customization(slide_index, plot_uid):
    render_slide(slide_index)  # Rerender the main slide view with updated parameters
    # Update the thumbnail for the current slide to reflect changes
    sidebar.children = tuple(
        create_thumbnail(i) if i == slide_index else sidebar.children[i] for i in range(len(slides))
    )

# Update plot parameters in the slides list and refresh preview
def update_plot_params(slide_index, plot_uid, param_name, param_value):
    plot = next(plot for plot in slides[slide_index]['plots'] if plot['uid'] == plot_uid)
    plot['params'][param_name] = param_value
    update_plot_preview(slide_index, plot_uid)

# Initialize sidebar with thumbnails for each slide, each with a slide selection checkbox
def create_thumbnail(slide_index):
    thumbnail = widgets.Output()
    with thumbnail:
        fig, axs = plt.subplots(2, 2, figsize=(3, 3))
        position_map = {'top-left': (0, 0), 'top-right': (0, 1), 'bottom-left': (1, 0), 'bottom-right': (1, 1)}
        for plot in slides[slide_index]['plots']:
            pos = position_map[plot['position']]
            plot_graph(axs[pos], **plot['params'])
        plt.tight_layout()
        plt.close(fig)
        display(fig)

    # Add checkbox to select this slide
    checkbox = widgets.Checkbox(description=f"Select Slide {slide_index + 1}", value=False, layout=widgets.Layout(width='auto'))
    checkbox.observe(lambda change: show_slide(change, slide_index), names='value')
    slide_checkboxes.append(checkbox)
    return widgets.VBox([thumbnail, checkbox], layout=widgets.Layout(margin='5px'))

# Show slide when selected in the sidebar
def show_slide(change, slide_index):
    global selected_slide_index
    if change['new']:
        # Uncheck all other slide checkboxes to enforce single selection
        for cb in slide_checkboxes:
            if cb != change['owner']:
                cb.value = False
        selected_slide_index = slide_index
        render_slide(slide_index)

# Display main layout with sidebar and main slide display
def display_main_layout():
    global slide_checkboxes
    slide_checkboxes = []  # Reset slide checkboxes
    sidebar.children = [create_thumbnail(i) for i in range(len(slides))]
    render_slide(selected_slide_index if selected_slide_index is not None else 0)  # Display the initial slide
    main_layout = widgets.VBox([widgets.HBox([sidebar, main_slide_display, customization_window]), button_box])
    display(main_layout)  # Ensure all sections are visible

# Initial display
display_main_layout()
